# Library Import

In [ ]:
# --- Install required packages (run once in Colab) ---
!pip install geopy
!pip install chardet
!pip install meteostat==1.6.8
!pip install sktime
!pip install -U scikit-learn
!pip install nbformat

# --- Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- Core libraries ---
import pandas as pd
import numpy as np
import statsmodels.api as stats

# --- Visualization ---
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import colorcet as cc
from matplotlib.ticker import FuncFormatter

sns.set_palette(cc.glasbey_cool)  # Set the palette for the notebook

# --- Statsmodels ---
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.graphics.gofplots import qqplot
from statsmodels.stats.outliers_influence import variance_inflation_factor

# --- Geopy for geocoding ---
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from geopy.geocoders import Nominatim

# --- Meteostat for weather data ---
import meteostat
from meteostat import Daily

# --- Scikit-learn ---
from sklearn import metrics, tree, linear_model
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.impute import KNNImputer
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    f1_score,
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
    r2_score,
    accuracy_score,
    confusion_matrix,
    classification_report,
    root_mean_squared_error,
)
from sklearn.model_selection import (
    GridSearchCV,
    TimeSeriesSplit,
    RandomizedSearchCV,
    BaseCrossValidator,
    cross_val_score,
    train_test_split,
)
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, StandardScaler, TargetEncoder
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.utils.validation import column_or_1d

# --- Sktime (time series forecasting) ---
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.exp_smoothing import ExponentialSmoothing

# --- Other standard libs and utilities ---
import calendar
import os
import re
import time
from collections import defaultdict
from datetime import datetime
from typing import Tuple

# --- Other third-party libs ---
import nbformat
import chardet
import colorcet
import matplotlib
import numpy
import pandas
import scipy
import seaborn
import shap
import statsmodels
from scipy.stats import skew

seed=42

>
### __Define User Functions__

#### __Retrieve Lat and Lon from Postcodes__

In [ ]:
def get_coordinates_from_postcode(postcode, max_retries=3):
    """
    This function retrieves the geographic coordinates (latitude and longitude)
    for a given Spanish postcode using the Nominatim geocoding service.

    It attempts the request up to a specified number of retries in case of
    timeouts or connection errors, with a delay between each retry to respect
    the Nominatim usage policy.

    Parameters:
    - postcode (str): The postcode for which coordinates are to be retrieved.
    - max_retries (int): Maximum number of retry attempts in case of failure
      (default is 3).

    Returns:
    - A tuple containing (latitude, longitude) as floats.
      Returns (None, None) if the location could not be retrieved.
    """
    geolocator = Nominatim(user_agent="postcode_locator", timeout=5) # Timeout after waiting 5 secs.
    attempt = 0

    while attempt < max_retries:
        try:
            print(f"Getting coordinates for: {postcode}")
            location = geolocator.geocode(f"{postcode}, country", addressdetails=False)

            if location:
                time.sleep(1)  # Respect usage policy.
                return location.latitude, location.longitude
            else:
                return None, None

        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"Retry {attempt + 1} for postcode {postcode} due to error: {e}")
            attempt += 1
            time.sleep(2) # Wait before retrying.

    return None, None # If all retries fail.

In [ ]:
def get_daily_temp_normals(
    station_id='08221',
    start_year=1991,
    end_year=2020,
    k_neighbors=5
):
    """
    Retrieves daily reference normals (1991–2020 by default) for tmax, tmin, and tavg
    using Meteostat data. Applies KNN imputation for missing values.

    Parameters:
    - station_id (str): Meteostat station ID (default: _____)
    - start_year (int): Start year of the period (default: 1991)
    - end_year (int): End year of the period (default: 2020)
    - k_neighbors (int): Number of neighbors for KNN imputation (default: 5)

    Returns:
    - dict: Dictionary with keys 'ref_tmax', 'ref_tmin', 'ref_tavg', each mapping
            day-of-year (1–365) to the corresponding normal temperature.
    """

    # Fetch data from metropolitan area, the airport is considered to be
    # a good representation of weather data for the Community.
    # Set start and end dates for collection.
    start = datetime(start_year, 1, 1)
    end = datetime(end_year, 12, 31)

    print("Fetching data...")
    data = Daily(station_id, start, end).fetch()
    data = data[['tmax', 'tmin', 'tavg']].copy()

    # Impute missing values using KNN.
    print(f"Missing values before imputation:\n{data.isna().sum()}")

    if data.isna().any().any():
        print("Imputing missing values using KNN...")
        imputer = KNNImputer(n_neighbors=k_neighbors)
        data[['tmax', 'tmin', 'tavg']] = imputer.fit_transform(data[['tmax', 'tmin', 'tavg']])
        print("Missing values imputed.")
    else:
        print("No missing values found.")

    # Select data which is NOT February 29. This will leave gaps for leap years
    # which will need to be imputed in the dataset.
    data = data[~((data.index.month == 2) & (data.index.day == 29))]

    # Add day-of-year column (now guaranteed to be 1–365).
    data['day_of_year'] = data.index.dayofyear
    data['day_of_year'] = data['day_of_year'].apply(lambda x: 365 if x == 366 else x)

    # Group by day-of-year.
    print("Calculating daily normals...")
    daily_normals = data.groupby('day_of_year').mean().reindex(range(1, 366))  # Ensure all 365 days.

    # Format results to map.
    result = {
        'ref_tmax': daily_normals['tmax'].round(2).to_dict(),
        'ref_tmin': daily_normals['tmin'].round(2).to_dict(),
        'ref_tavg': daily_normals['tavg'].round(2).to_dict()
    }

    return result

#### __One-hot Encode DataFrame Columns__

In [ ]:
def one_hot_encode_columns(df, columns, prefix=True, drop_first=False, keep_original=False):
    """
    One-hot encodes multiple columns that are of object type, with an option to keep the original columns.

    Raises an error if any specified column is not of object or categorical type.

    Parameters:
    - df: the DataFrame which contains the relevant columns.
    - columns: list of column names to one-hot encode.
    - prefix: bool, if True adds the column name as a prefix to dummy columns;
              default is True.
    - drop_first: bool, if True drops the first category to avoid multicollinearity.
    - keep_original: bool, if True retains the original columns; default is False.

    Returns:
    - df: DataFrame with one-hot encoded columns added, and optionally the original columns kept.
    """
    df = df.copy()

    for col in columns:
        # Ensure the column is of object type.
        if not (df[col].dtype == 'object' or pd.api.types.is_categorical_dtype(df[col])):
            raise ValueError(f"Column '{col}' is not of object type and cannot be one-hot encoded.")

        # One-hot encode the column.
        dummies = pd.get_dummies(df[col], prefix=col if prefix else None, drop_first=drop_first)

        # Concatenate the one-hot encoded columns to the DataFrame.
        if keep_original:
            df = pd.concat([df, dummies], axis=1)  # Keep the original column.
        else:
            df = pd.concat([df.drop(columns=[col]), dummies], axis=1)  # Drop the original column.

    return df

#### __Add Cyclical Encoding for DataFrame Temporal Variables__

In [ ]:
def add_cyclical_encoding(df, columns_periods, drop_original=True):
    """
    Applies cyclical encoding (sine and cosine) to specified columns in a
    DataFrame. Should be used on variables such as day of week which are
    inherently cyclical. Variables should all be ordered

    Parameters:
    - df: DataFrame which contains the variables to be transformed.
    - columns_periods: dict of column names and their respective periods.
                       e.g., {'month': 12, 'day': 31}
    - drop_original: bool, if True drops the original columns after encoding.

    Returns:
    - df: DataFrame with new cyclical features added.
    """
    df = df.copy()

    for col, period in columns_periods.items():
        sin_col = f"{col}_sin"
        cos_col = f"{col}_cos"

        df[sin_col] = np.sin(2 * np.pi * df[col] / period)
        df[cos_col] = np.cos(2 * np.pi * df[col] / period)

        if drop_original:
            df.drop(columns=[col], inplace=True)

    return df

#### __Determine Best TimeSeries Split__

In [ ]:
def evaluate_time_series_split_masked(model, df, y_col, split_range=range(3, 11), scoring='neg_mean_squared_error'):
    """
    Evaluate a time series regression model using TimeSeriesSplit with varying numbers of splits.

    Parameters:
    - model: A scikit-learn compatible regressor instance (e.g., XGBRegressor(), LGBMRegressor()).
    - df (pd.DataFrame): Full DataFrame containing features and target, indexed by datetime.
    - y_col (str): Name of the target column.
    - split_range (range): Range of split counts to evaluate (default is range(3, 11)).
    - scoring (str): Scoring metric to use (default is 'neg_mean_squared_error').

    Returns:
    - mean_scores (list): List of mean scores for each split count.
    - std_scores (list): List of standard deviations of scores for each split count.
    """

    if not pd.api.types.is_datetime64_any_dtype(df.index):
        raise ValueError("The DataFrame index must be of datetime type.")

    # Extract feature set (X) and target variable (y)
    X = df.drop(columns=[y_col])
    y = df[y_col]

    # Get unique sorted dates for TimeSeriesSplit
    unique_dates = df.index.unique().sort_values()

    mean_scores = []
    std_scores = []

    for n_splits in split_range:
        # Initialize TimeSeriesSplit with the current number of splits
        tscv = TimeSeriesSplit(n_splits=n_splits)

        fold_scores = []

        # Perform cross-validation with time-based splits
        for train_idx, test_idx in tscv.split(unique_dates):
            train_dates = set(unique_dates[train_idx])
            test_dates = set(unique_dates[test_idx])

            # Mask based on whether index is in train or test dates
            train_mask = df.index.map(train_dates.__contains__)
            test_mask = df.index.map(test_dates.__contains__)

            # Split the data
            X_train, X_test = X[train_mask], X[test_mask]
            y_train, y_test = y[train_mask], y[test_mask]

            # Evaluate the model
            model.fit(X_train, y_train)
            score = cross_val_score(model, X_test, y_test, cv=tscv, scoring=scoring).mean()  # Evaluating only test set
            fold_scores.append(score)

        # Convert negative MSE to positive and calculate mean and std
        mean_mse = -np.mean(fold_scores)  # We convert back to positive MSE
        std_mse = np.std(fold_scores)

        mean_scores.append(mean_mse)
        std_scores.append(std_mse)

        print(f"{n_splits} splits: Mean MSE = {mean_mse:.4f}, Std = {std_mse:.4f}")

    # Plot the results
    plt.figure(figsize=(8, 5))
    plt.plot(split_range, mean_scores, marker='o', label='Mean MSE', color='blue')
    plt.fill_between(split_range,
                     [m - s for m, s in zip(mean_scores, std_scores)],
                     [m + s for m, s in zip(mean_scores, std_scores)],
                     color='gray', alpha=0.2, label='±1 Std Dev')
    plt.title('Evaluation of TimeSeriesSplit for Varying Split Counts', fontsize=14)
    plt.xlabel('Number of Splits', fontsize=12)
    plt.ylabel('Mean MSE', fontsize=12)
    plt.legend()
    plt.grid(True)
    plt.show()

    return mean_scores, std_scores

#### __Create TimeSeries Splits__

In [ ]:
def get_timeseries_splits_with_masking(df, y, n_splits=5):
    """
    Splits the data into training and testing sets using TimeSeriesSplit with datetime index.
    Applies the same masking logic as used in the Random Forest model, ensuring no overlap
    between the train and test sets. Prints the train/test indices for each fold.

    Parameters:
    - df (DataFrame): The DataFrame containing the feature variables and target variable.
      The index must be of datetime type, and lag features should already be included.
    - y (str): The target variable (dependent variable).
    - n_splits (int, optional): The number of splits for TimeSeriesSplit. Default is 5.

    Returns:
    - list of tuples: Each tuple contains:
      - X_train: Training feature data
      - X_test: Testing feature data
      - y_train: Training target data
      - y_test: Testing target data
    """
    if not pd.api.types.is_datetime64_any_dtype(df.index):
        raise ValueError("The DataFrame index must be of datetime type.")

    feature_cols = [col for col in df.columns if col != y]  # Exclude the target variable.
    X = df[feature_cols]
    y_series = df[y]

    # Get unique sorted dates for TimeSeriesSplit
    unique_dates = df.index.unique().sort_values()

    splits = []

    # Initialize TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # Generate train/test splits
    for fold, (train_idx, test_idx) in enumerate(tscv.split(unique_dates)):
        print(f"Fold {fold + 1}")

        # Get train and test data by dates (unique sorted)
        train_dates = set(unique_dates[train_idx])
        test_dates = set(unique_dates[test_idx])

        # Mask based on whether index is in train or test dates
        train_mask = df.index.map(train_dates.__contains__)
        test_mask = df.index.map(test_dates.__contains__)

        # Create the final train and test sets based on these masks
        X_train, X_test = X[train_mask], X[test_mask]
        y_train, y_test = y_series[train_mask], y_series[test_mask]

        # Check for overlap of dates between train and test
        overlap_dates = np.intersect1d(df.index[train_mask], df.index[test_mask])

        if len(overlap_dates) > 0:
            print(f"Warning: Duplicate dates found between train and test "
                  f"for Fold {fold + 1} of {n_splits}: {overlap_dates}")
        else:
            print(f"No duplicates between train and test for Fold {fold + 1} of {n_splits}")
            print("Proceeding with train-test split...")

        # Store the splits
        splits.append((X_train, X_test, y_train, y_test))

        # Print out the train-test split details
        print(f"Train indices: {train_idx}")
        print(f"Test indices: {test_idx}")
        print("-" * 30)

    return splits

#### __Check for Overfitting on Last Fold of Boosted Regression Model__

In [ ]:
def check_overfitting_on_last_fold_masked(df, y_col, n_splits, model):
    """
    Checks for overfitting by training on the last fold of TimeSeriesSplit and comparing
    R² scores on train and test sets.

    Parameters:
    - df (pd.DataFrame): Full DataFrame containing features and target, indexed by datetime.
    - y_col (str): Name of the target column.
    - n_splits (int): Number of TimeSeriesSplit folds.
    - model: A model instance (e.g., best XGBoost model).

    Returns:
    - train_r2: R² on training set.
    - test_r2: R² on test set.
    """

    if not pd.api.types.is_datetime64_any_dtype(df.index):
        raise ValueError("The DataFrame index must be of datetime type.")

    # Extract feature set (X) and target variable (y)
    X = df.drop(columns=[y_col])
    y = df[y_col]

    # Get unique sorted dates
    unique_dates = df.index.unique().sort_values()

    # Initialize TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # Track the last fold's train and test indices
    for fold, (train_idx, test_idx) in enumerate(tscv.split(unique_dates)):
        pass  # Loop until the last fold, don't train yet

    # Get the last fold's train and test date sets
    train_dates = set(unique_dates[train_idx])
    test_dates = set(unique_dates[test_idx])

    # Mask based on whether index is in train or test dates
    train_mask = df.index.map(train_dates.__contains__)
    test_mask = df.index.map(test_dates.__contains__)

    # Create train and test datasets based on the mask
    X_train, X_test = X[train_mask], X[test_mask]
    y_train, y_test = y[train_mask], y[test_mask]

    # Fit the model on the training data
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)

    # Calculate R² for both train and test sets

    train_r2 = r2_score(y_train, train_pred)
    test_r2 = r2_score(y_test, test_pred)

    # Output the R² scores and check for overfitting
    print(f"\nOverfitting Check (Last Fold):")
    print(f"Train R²: {train_r2:.4f}")
    print(f"Test  R²: {test_r2:.4f}")

    if train_r2 - test_r2 > 0.2:
        print("⚠️ Model likely overfitting. Consider simplifying or regularizing.")
    else:
        print("✅ No significant overfitting detected.")

    return train_r2, test_r2

#### __Generate Static Calendar Feature__

In [ ]:
def generate_static_calendar_features(year: int) -> pd.DataFrame:
    # Generate daily date range for the given year
    future_dates = pd.date_range(f"{year}-01-01", f"{year}-12-31", freq='D')

    df = pd.DataFrame({"date": future_dates})

    # Extract basic calendar components
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month.astype(np.int32)
    df["day"] = df["date"].dt.day
    df["day_of_week"] = df["date"].dt.dayofweek  # Monday=0
    df["day_of_year"] = df["date"].dt.dayofyear

    # Add cyclical features for month and weekday
    df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
    df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)
    df["day_of_week_sin"] = np.sin(2 * np.pi * df["day_of_week"] / 7)
    df["day_of_week_cos"] = np.cos(2 * np.pi * df["day_of_week"] / 7)

    return df

#### __Create Forecast Panel__

In [ ]:
def create_forecast_panel(X, df_unpaneled, station_code='sales_location',
                               one_hot_columns=['province', 'town/city', 'company_code']):
    """
    Create forecast panel with one row per date per sales station by joining
    df_unpaneled with one-hot encoded station columns extracted exactly from X.

    Parameters:
    - X: pd.DataFrame
        Training DataFrame containing one-hot encoded columns and station_code.
    - df_unpaneled: pd.DataFrame
        DataFrame with daily weather stats indexed by date.
    - station_code: str, default 'sales_location'
        Column name identifying sales locations in X.
    - one_hot_columns: list of str
        Exact one-hot encoded column names to extract from X.

    Returns:
    - forecast_panel: pd.DataFrame
        One row per date per station with all features combined.
    """
    # Extract station info columns exactly as encoded in X
    station_lookup = X[[station_code] + one_hot_columns].drop_duplicates(subset=[station_code]).reset_index(drop=True)

    # Add dummy key to enable cross join
    df_unpaneled = df_unpaneled.copy()
    station_lookup = station_lookup.copy()
    df_unpaneled['key'] = 1
    station_lookup['key'] = 1

    # Cross join daily scenarios with station lookup
    df_paneled = pd.merge(df_unpaneled, station_lookup, on='key').drop('key', axis=1)

    return df_paneled

#### __Create Historical Climate Data__

In [ ]:
def get_daily_tmax_normals_city(station_id='08221', k_neighbors=5):
    """
    This function retrieves the daily values for the city for the relevant 30 year
    Period 1991-2020 and calculates daily reference normals using the Meteostat
    package. It checks for missing climate values and imputes any missing values
    with KNN imputation, returning a dictionary mapping day-of-year (1–365) to
    normal tmax.

    Parameters:
    - station_id (optional, str): The identifier for the weather station of choice;
                                  defaults to the city.
    - k_neighbors (optional, int): The k value for KNN imputation; defaults to 5.

    Returns:
    - Daily reference normals for 1991-2020.
    """
    # Fetch data from the city, the airport is considered to be
    # a good representation of weather data for the Community.
    # Set start and end dates for collection.
    start = datetime(1991, 1, 1)
    end = datetime(2020, 12, 31)

    print("Fetching data...")
    data = Daily(station_id, start, end).fetch()
    data = data[['tmax']].copy()

    # Impute missing values using KNN.
    missing_count = data['tmax'].isna().sum()
    print(f"Missing days before imputation: {missing_count}")

    if data['tmax'].isna().sum() > 0:
        print("Imputing missing tmax values before further processing...")
        imputer = KNNImputer(n_neighbors=k_neighbors)
        data[['tmax']] = imputer.fit_transform(data[['tmax']])
        print("Missing values imputed using KNN.")
    else:
        print("No missing tmax values found.")

    # Select data which is NOT February 29. This will leave gaps for leap years
    # which will need to be imputed in the dataset.
    data = data[~((data.index.month == 2) & (data.index.day == 29))]

    # Add day-of-year column (now guaranteed to be 1–365)
    data['day_of_year'] = data.index.dayofyear
    data['day_of_year'] = data['day_of_year'].replace(366, 365)

    # Group by day-of-year.
    print("Calculating daily normals...")
    daily_normals = data.groupby('day_of_year').mean()
    daily_normals = daily_normals.reindex(range(1, 366))  # Ensure all 365 days.

    # Final formatting
    daily_normals = daily_normals.rename(columns={'tmax': 'ref_tmax'})
    daily_normals.index.name = 'day_of_year'

    return daily_normals['ref_tmax'].round(2).to_dict()

In [ ]:
def get_daily_temp_normals(
    station_id='08221',
    start_year=1991,
    end_year=2020,
    k_neighbors=5
):
    """
    Retrieves daily reference normals (1991–2020 by default) for tmax, tmin, and tavg
    using Meteostat data. Applies KNN imputation for missing values.

    Parameters:
    - station_id (str): Meteostat station ID (default: ____)
    - start_year (int): Start year of the period (default: 1991)
    - end_year (int): End year of the period (default: 2020)
    - k_neighbors (int): Number of neighbors for KNN imputation (default: 5)

    Returns:
    - dict: Dictionary with keys 'ref_tmax', 'ref_tmin', 'ref_tavg', each mapping
            day-of-year (1–365) to the corresponding normal temperature.
    """

    # Fetch data from the city, the airport is considered to be
    # a good representation of weather data for the Community.
    # Set start and end dates for collection.
    start = datetime(start_year, 1, 1)
    end = datetime(end_year, 12, 31)

    print("Fetching data...")
    data = Daily(station_id, start, end).fetch()
    data = data[['tmax', 'tmin', 'tavg']].copy()

    # Impute missing values using KNN.
    print(f"Missing values before imputation:\n{data.isna().sum()}")

    if data.isna().any().any():
        print("Imputing missing values using KNN...")
        imputer = KNNImputer(n_neighbors=k_neighbors)
        data[['tmax', 'tmin', 'tavg']] = imputer.fit_transform(data[['tmax', 'tmin', 'tavg']])
        print("Missing values imputed.")
    else:
        print("No missing values found.")

    # Select data which is NOT February 29. This will leave gaps for leap years
    # which will need to be imputed in the dataset.
    data = data[~((data.index.month == 2) & (data.index.day == 29))]

    # Add day-of-year column (now guaranteed to be 1–365).
    data['day_of_year'] = data.index.dayofyear
    data['day_of_year'] = data['day_of_year'].apply(lambda x: 365 if x == 366 else x)

    # Group by day-of-year.
    print("Calculating daily normals...")
    daily_normals = data.groupby('day_of_year').mean().reindex(range(1, 366))  # Ensure all 365 days.

    # Format results to map.
    result = {
        'ref_tmax': daily_normals['tmax'].round(2).to_dict(),
        'ref_tmin': daily_normals['tmin'].round(2).to_dict(),
        'ref_tavg': daily_normals['tavg'].round(2).to_dict()
    }

    return result

In [ ]:
def get_historical_daily_temps(station_id='08221', start_year=1991, end_year=2022, k_neighbors=5):
    """
    Fetches and cleans historical daily temperature data (tmax, tmin, tavg) for a given station.

    - Removes February 29 (leap day)
    - Applies KNN imputation to fill missing temperature values
    - Adds `date`, `day_of_year`, and `year` columns

    Parameters:
        station_id (str): Meteostat station ID (default: ____)
        start_year (int): Start year for fetching historical data
        end_year (int): End year for fetching historical data
        k_neighbors (int): Number of neighbors for KNN imputation

    Returns:
        pd.DataFrame: Cleaned historical weather data with columns: ['tmax', 'tmin', 'tavg', 'date', 'day_of_year', 'year']
    """

    start = datetime(start_year, 1, 1)
    end = datetime(end_year, 12, 31)

    print("Fetching historical data...")
    data = Daily(station_id, start, end).fetch()[['tmax', 'tmin', 'tavg']].copy()

    # Check how many February 29th dates there are before removal
    print("February 29th count before removal:", len(data[(data.index.month == 2) & (data.index.day == 29)]))

    # Remove leap day (Feb 29).
    data = data[~((data.index.month == 2) & (data.index.day == 29))]

    # Check how many February 29th dates remain after removal
    print("February 29th count after removal:", len(data[(data.index.month == 2) & (data.index.day == 29)]))

    # Impute missing values
    if data.isna().any().any():
        print("Imputing missing values using KNN...")
        imputer = KNNImputer(n_neighbors=k_neighbors)
        data[['tmax', 'tmin', 'tavg']] = imputer.fit_transform(data[['tmax', 'tmin', 'tavg']])
        print("Missing values imputed.")

    data = data.round(2)
    data['date'] = data.index
    data['day_of_year'] = data.index.dayofyear
    data['day_of_year'] = data['day_of_year'].replace(366, 365)

    # Add year column
    data['year'] = data.index.year
    data.reset_index(drop=True, inplace=True)

    return data

In [ ]:
def get_daily_tmax_normals_mean_std(df, station_id='08221', k_neighbors=5):
    """
    Retrieve daily maximum temperature (tmax) reference normals and calculate
    historical deviation statistics for the city using Meteostat data (1991–2020).

    This function:
    - Downloads daily historical tmax data for the station.
    - Removes leap day (Feb 29) to standardize to a 365-day year.
    - Imputes missing values using KNN imputation.
    - Computes daily reference normals by averaging tmax across all years per day-of-year.
    - Calculates mean and standard deviation of daily deviations (actual - normal).

    Parameters:
    - station_id (str): Meteostat station ID (default: _____)
    - k_neighbors (int): Number of neighbors for KNN imputation (default: 5)

    Returns:
    - df (pd.DataFrame): DataFrame with a DatetimeIndex to determine the time window.
    - ref_tmax_dict: dict mapping day_of_year (1–365) → average tmax
    - mean_deviation: float, mean of all daily temperature deviations
    - std_deviation: float, std dev of all daily temperature deviations
    """
    # Fetch data from the city. The airport is considered to be
    # a good representation of weather data for the Community.

    # Check DataFrame has datetime index.
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("Input df must have a DatetimeIndex.")

    # Derive dynamic time window for data collection.
    end_date = df.index.min()
    start_date = end_date - pd.DateOffset(years=30)
    print(f"Fetching normals from {start_date.date()} to {end_date.date()}")

    # Fetch data from Meteostat.
    print("Fetching data...")
    data = Daily(station_id, start_date, end_date).fetch()
    data = data[['tmax']].copy()

    # Impute missing values using KNN.
    if data['tmax'].isna().sum() > 0:
        print(f"Missing values before imputation: {data['tmax'].isna().sum()}")
        imputer = KNNImputer(n_neighbors=k_neighbors)
        data[['tmax']] = imputer.fit_transform(data[['tmax']])
        print("Missing values imputed.")

    # Select data which is NOT February 29. This will leave gaps for leap years
    # which will need to be imputed in the dataset.
    data = data[~((data.index.month == 2) & (data.index.day == 29))]

    # Add day of year.
    data['day_of_year'] = data.index.dayofyear

    # Calculate daily normals
    print("Calculating daily normals...")
    daily_normals = data.groupby('day_of_year')['tmax'].mean().reindex(range(1, 366))
    daily_normals.name = 'ref_tmax'

    # Merge normals back for deviation calculation.
    data = data.merge(daily_normals, left_on='day_of_year', right_index=True)
    data['temp_deviation'] = data['tmax'] - data['ref_tmax']

    # Calculate statistics.
    mean_deviation = data['temp_deviation'].mean()
    std_deviation = data['temp_deviation'].std()

    print(f"Mean deviation: {mean_deviation:.2f}, Std deviation: {std_deviation:.2f}")

    return daily_normals.round(2).to_dict(), mean_deviation, std_deviation

In [ ]:
# Circular mean for wind direction
def circular_mean_deg(series):
    radians = np.deg2rad(series.dropna())
    sin_sum = np.sum(np.sin(radians))
    cos_sum = np.sum(np.cos(radians))
    mean_angle = np.arctan2(sin_sum, cos_sum)
    return np.rad2deg(mean_angle) % 360

def mod_func(x):
    return x.mode().iloc[0] if not x.mode().empty else np.nan

def prcp_stats(x):
    rain_days = x[x > 0]
    prob = len(rain_days) / len(x)
    if prob > 0:  # If rain has a probability greater than zero calculate stats otherwise rain stats should be 0.0 (prevents 0 days skewing aggregate calculations)
        return pd.Series({
            'prcp_rain_prob': prob,
            'prcp_mean_if_rain': rain_days.mean(),
            'prcp_max_if_rain': rain_days.max(),
            'prcp_mode_if_rain': mod_func(rain_days)
        })
    else:
        return pd.Series({
            'prcp_rain_prob': 0.0,
            'prcp_mean_if_rain': 0.0,
            'prcp_max_if_rain': 0.0,
            'prcp_mode_if_rain': 0.0
        })

def get_daily_variability(
    station_id='08221',
    start_year=1991,
    end_year=2022,
    k_neighbors=5
):
    """
    Retrieves daily reference normals (1991–2020 by default) for tmax, tmin, and tavg
    using Meteostat data. Applies KNN imputation for missing values.

    Parameters:
    - station_id (str): Meteostat station ID (default: ___)
    - start_year (int): Start year of the period (default: 1991)
    - end_year (int): End year of the period (default: 2020)
    - k_neighbors (int): Number of neighbors for KNN imputation (default: 5)

    Returns:
    - dict: Dictionary with keys 'ref_tmax', 'ref_tmin', 'ref_tavg', each mapping
            day-of-year (1–365) to the corresponding normal temperature.
    """

    # Fetch data from the city the airport is considered to be
    # a good representation of weather data for the Community.
    # Set start and end dates for collection.
    start = datetime(start_year, 1, 1)
    end = datetime(end_year, 12, 31)

    print("Fetching data...")
    data = Daily(station_id, start, end).fetch()
    data = data[['tmax', 'tmin', 'tavg', 'pres', 'prcp', 'wspd', 'wdir']].copy()

    # Separate imputation to preserve prcp = 0.0
    to_impute = ['tmax', 'tmin', 'tavg', 'pres', 'wspd', 'wdir']
    if data[to_impute + ['prcp']].isna().any().any():
        print("Imputing missing values using KNN...")

        # Impute temperature and pressure only
        imputer = KNNImputer(n_neighbors=k_neighbors)
        data[to_impute] = imputer.fit_transform(data[to_impute])

        # Handle precipitation imputation separately so that 0.0 values are retained.
        prcp_missing = data['prcp'].isna()
        if prcp_missing.any():
            prcp_imputer = KNNImputer(n_neighbors=k_neighbors)
            # Only fit on rows where prcp is not NaN
            prcp_features = data[to_impute + ['prcp']]
            imputed = prcp_imputer.fit_transform(prcp_features)
            data['prcp'] = imputed[:, -1]  # last column is prcp

        print("Missing values imputed.")
    else:
        print("No missing values found.")

    # Check for missing values after imputation
    print(f"Missing values after imputation:\n{data.isna().sum()}")

    # Check how many February 29th dates there are before removal
    print("February 29th count before removal:", len(data[(data.index.month == 2) & (data.index.day == 29)]))

    # Remove leap day (Feb 29).
    data = data[~((data.index.month == 2) & (data.index.day == 29))]

    # Check how many February 29th dates remain after removal
    print("February 29th count after removal:", len(data[(data.index.month == 2) & (data.index.day == 29)]))

    # Impute missing values using KNN.
    print(f"Missing values before imputation:\n{data.isna().sum()}")

    # Add day-of-year column (now guaranteed to be 1–365).
    data['day_of_year'] = data.index.dayofyear
    data['day_of_year'] = data['day_of_year'].replace(366, 365)

    # Group by day-of-year to calculate daily min, max, mean, and mode.
    print("Calculating temperature and pressure normals...")
    core_stats = data.groupby('day_of_year').agg({
        'tmax': ['min', 'mean', 'max', mod_func],
        'tmin': ['min', 'mean', 'max', mod_func],
        'tavg': ['min', 'mean', 'max', mod_func],
        'pres': ['min', 'mean', 'max', mod_func],
        'wspd': ['min', 'mean', 'max', mod_func],
        'wdir': [circular_mean_deg]
    })

    # Aggregate precipitation separately
    print("Calculating rain probability and rain-only stats...")
    rain_stats = data.groupby('day_of_year')['prcp'].apply(prcp_stats).unstack()

    # Merge all daily climate stats
    print("Merging all daily stats...")
    final_data = pd.concat([core_stats, rain_stats], axis=1)

    print("Final data columns:", final_data.columns)
    print("Number of columns:", len(final_data.columns))

    # Flatten MultiIndex columns
    final_data.columns = [
        'tmax_min', 'tmax_mean', 'tmax_max', 'tmax_mode',
        'tmin_min', 'tmin_mean', 'tmin_max', 'tmin_mode',
        'tavg_min', 'tavg_mean', 'tavg_max', 'tavg_mode',
        'avg_pressure_min', 'avg_pressure_mean', 'avg_pressure_max', 'avg_pressure_mode',
        'precip_rain_prob', 'precip_mean_if_rain', 'precip_max_if_rain', 'precip_mode_if_rain',
        'wind_speed_min', 'wind_speed_mean', 'wind_speed_max', 'wind_speed_mode',
        'wind_dir_mean'
    ]
    return final_data

#### __Create Climate Scenarios__

In [ ]:
def generate_future_scenarios(temp_stats_hist, year=2023):
    # Generate full-year date range
    future_dates = pd.date_range(f"{year}-01-01", f"{year}-12-31", freq='D')

    # Build scenario rows using historical stats
    scenario_records = []
    for date in future_dates:
        day = date.timetuple().tm_yday
        row = temp_stats_hist.loc[day]

        scenario_records.append({
            'date': date,
            'day_of_year': day,

            # Temperature
            'tmax_min': row['tmax_min'],
            'tmax_mean': row['tmax_mean'],
            'tmax_max': row['tmax_max'],
            'tmax_mode': row['tmax_mode'],

            'tmin_min': row['tmin_min'],
            'tmin_mean': row['tmin_mean'],
            'tmin_max': row['tmin_max'],
            'tmin_mode': row['tmin_mode'],

            'tavg_min': row['tavg_min'],
            'tavg_mean': row['tavg_mean'],
            'tavg_max': row['tavg_max'],
            'tavg_mode': row['tavg_mode'],

            # Pressure
            'avg_pressure_min': row['avg_pressure_min'],
            'avg_pressure_mean': row['avg_pressure_mean'],
            'avg_pressure_max': row['avg_pressure_max'],
            'avg_pressure_mode': row['avg_pressure_mode'],

            # Precipitation
            'precip_rain_prob': row['precip_rain_prob'],
            'precip_mean_if_rain': row['precip_mean_if_rain'],
            'precip_max_if_rain': row['precip_max_if_rain'],
            'precip_mode_if_rain': row['precip_mode_if_rain'],

            # Wind speed
            'wind_speed_min': row['wind_speed_min'],
            'wind_speed_mean': row['wind_speed_mean'],
            'wind_speed_max': row['wind_speed_max'],
            'wind_speed_mode': row['wind_speed_mode'],

            # Wind direction (fixed for the day)
            'wind_dir_mean': row['wind_dir_mean']
        })

    # Create and return the merged DataFrame
    future_climate_scenarios = pd.DataFrame(scenario_records)
    return future_climate_scenarios

#### __Monte Carlo Sampling Function__

In [ ]:
def sample_uniform_monte(min_val, max_val, size=1):
    """
    This function randomly a uniform distribution between min and max for every day.
    It will repeat sampling nultiple times to generate a distribution of possible
    outcomes - capturing uncertainty. Number of samples is set by n_simulations.
    """
    return np.random.uniform(min_val, max_val, size)

#### __Perform Model Forecast with Monte Carlo Simulation__

In [ ]:
def run_simulation(future_climate_scenarios, forecast_df_panel, model, sampler, n_simulations=100):
    # Get the expected features from the model
    try:
        expected_features = model.feature_names_in_
    except AttributeError:
        raise ValueError("Model must have 'feature_names_in_' attribute (like most sklearn models).")

    # Extract which weather features are actually needed by the model
    weather_columns = {
        'avg_temp', 'min_temp', 'max_temp',
        'avg_pressure', 'wind_speed', 'wind_dir', 'precip'
    }
    weather_needed = sorted(set(expected_features) & weather_columns)

    if not weather_needed:
        raise ValueError("Model does not appear to use any weather features — are you sure that's correct?")

    # Prepare to collect all simulation results and maximum temperature metrics
    all_preds = []
    temp_sims = []

    for _ in range(n_simulations):
        simulation_df = forecast_df_panel.copy()

        # Sample only required weather columns
        weather_by_date = {}
        for _, row in future_climate_scenarios.iterrows():
            date = row['date']
            sampled = {}

            if 'avg_temp' in weather_needed:
                sampled['avg_temp'] = sampler(row['tavg_min'], row['tavg_max'])[0]
            if 'min_temp' in weather_needed:
                sampled['min_temp'] = sampler(row['tmin_min'], row['tmin_max'])[0]
            if 'max_temp' in weather_needed:
                sampled['max_temp'] = sampler(row['tmax_min'], row['tmax_max'])[0]
            if 'avg_pressure' in weather_needed:
                sampled['avg_pressure'] = sampler(row['avg_pressure_min'], row['avg_pressure_max'])[0]
            if 'wind_speed' in weather_needed:
                sampled['wind_speed'] = sampler(row['wind_speed_min'], row['wind_speed_max'])[0]
            if 'wind_dir' in weather_needed:
                sampled['wind_dir'] = (row['wind_dir_mean'] + np.random.normal(0, 5)) % 360
            if 'precip' in weather_needed:
                sampled['precip'] = sampler(row['precip_mean_if_rain'], row['precip_max_if_rain'])[0] \
                    if np.random.rand() < row['precip_rain_prob'] else 0.0

            weather_by_date[date] = sampled

        # Assign only required weather columns
        for feature in weather_needed:
            simulation_df[feature] = simulation_df['date'].map(lambda d: weather_by_date[d][feature])

        # Align features for prediction
        try:
            X = simulation_df[expected_features]
        except KeyError as e:
            missing = list(set(expected_features) - set(simulation_df.columns))
            raise ValueError(f"Missing required features for model prediction: {missing}")

        preds = model.predict(X)
        all_preds.append(preds)
        temp_sims.append(simulation_df['max_temp'].values)

    # Summarise predictions
    preds_array = np.array(all_preds)
    base_info = forecast_df_panel[['date', 'town/city']].reset_index(drop=True)
    forecast_summary = base_info.copy()
    forecast_summary['mean'] = preds_array.mean(axis=0)
    forecast_summary['low_80'] = np.percentile(preds_array, 10, axis=0)
    forecast_summary['up_80'] = np.percentile(preds_array, 90, axis=0)
    forecast_summary['low_95'] = np.percentile(preds_array, 2.5, axis=0)
    forecast_summary['up_95'] = np.percentile(preds_array, 97.5, axis=0)

    temp_array = np.array(temp_sims)  # shape: (n_simulations, n_rows)

    # Compute temperature stats
    forecast_summary['forecasted_max_temp'] = temp_array.mean(axis=0)
    forecast_summary['prob_exceed_28'] = (temp_array > 28).mean(axis=0)
    forecast_summary['conditional_mean_temp'] = np.where(
        (temp_array > 28).sum(axis=0) > 0,
        temp_array * (temp_array > 28),
        np.nan
    ).sum(axis=0) / np.maximum((temp_array > 28).sum(axis=0), 1)

    return forecast_summary


#### __Plot Permutation Feature Importance for XGBoosted Model__

In [ ]:
def tune_hgb_model_with_masking(df, y, params=None, n_splits=5, random_state=42):
    """
    Train an HistGradientBoostingRegressor using TimeSeriesSplit with time-aware masking,
    manually specifying hyperparameters (no grid search).

    Parameters:
    - df (DataFrame): Input dataframe with features and target. Must have a datetime index.
    - y (str): Target variable name.
    - params (dict, optional): Parameters for HistGradientBoostingRegressor.
    - n_splits (int): Number of CV splits.
    - random_state (int): Random seed.

    Returns:
    - model (HistGradientBoostingRegressor): Trained model on the full dataset.
    - cv_rmse (float): Average RMSE across CV folds.
    """

    if not pd.api.types.is_datetime64_any_dtype(df.index):
        raise ValueError("DataFrame index must be datetime type.")

    feature_cols = [col for col in df.columns if col != y]
    X = df[feature_cols]
    y_series = df[y]

    unique_dates = df.index.unique().sort_values()
    tscv = TimeSeriesSplit(n_splits=n_splits)

    params = params or {}
    params['random_state'] = random_state

    fold_rmse = []

    for train_idx, test_idx in tscv.split(unique_dates):
        train_dates = unique_dates[train_idx]
        test_dates = unique_dates[test_idx]

        train_mask = df.index.isin(train_dates)
        test_mask = df.index.isin(test_dates)

        X_train, y_train = X[train_mask], y_series[train_mask]
        X_test, y_test = X[test_mask], y_series[test_mask]

        model = HistGradientBoostingRegressor(**params)
        model.fit(X_train, y_train)

        preds = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        fold_rmse.append(rmse)

    cv_rmse = np.mean(fold_rmse)

    # Train final model on full data
    final_model = HistGradientBoostingRegressor(**params)
    final_model.fit(X, y_series)

    print(f"Cross-validated RMSE: {cv_rmse:.4f}")

    return final_model, fold_rmse, cv_rmse

#### __Perform HGBoosted Model Regression with Timeseries Split__

In [ ]:
def evaluate_hgb_performance_with_best_model_masked(df, y_col, n_splits=None, best_model=None):
    """
    Evaluates the HBBoost model using time series cross-validation and reports detailed metrics.

    Parameters:
    - df (pd.DataFrame): Full DataFrame containing features and target, indexed by datetime.
    - y_col (str): Name of the target column.
    - n_splits (int): Number of splits for TimeSeriesSplit.
    - best_model: Fitted HGBoost model (e.g., from GridSearchCV).

    Returns:
    - best_model: The trained model after evaluation.
    - fold_results (pd.DataFrame): Metrics for each fold.
    - avg_results (dict): Average metrics across folds.
    """

    if not pd.api.types.is_datetime64_any_dtype(df.index):
        raise ValueError("The DataFrame index must be of datetime type.")

    X = df.drop(columns=[y_col])
    y = df[y_col]

    unique_dates = df.index.unique().sort_values()
    tscv = TimeSeriesSplit(n_splits=n_splits)

    metrics_list = []

    for fold, (train_idx, test_idx) in enumerate(tscv.split(unique_dates)):
        print(f"\nFold {fold + 1} of {n_splits}")

        train_dates = set(unique_dates[train_idx])
        test_dates = set(unique_dates[test_idx])

        train_mask = df.index.map(train_dates.__contains__)
        test_mask = df.index.map(test_dates.__contains__)

        X_train, X_test = X[train_mask], X[test_mask]
        y_train, y_test = y[train_mask], y[test_mask]

        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)

        # Metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        mape = np.mean(np.abs((y_test - y_pred) / np.where(y_test == 0, 1, y_test))) * 100
        rmse_mae = rmse - mae
        rmse_mae_cent = (rmse_mae / rmse) * 100

        metrics_list.append({
            'Fold': f'Fold {fold + 1}',
            'MSE': mse,
            'RMSE': rmse,
            'R2': r2,
            'MAE': mae,
            'MAPE': mape,
            'RMSE - MAE': rmse_mae,
            'RMSE - MAE %': rmse_mae_cent
        })

    fold_results = pd.DataFrame(metrics_list)

    avg_results = {
        'Average MSE': fold_results['MSE'].mean(),
        'Average RMSE': fold_results['RMSE'].mean(),
        'Average R2': fold_results['R2'].mean(),
        'Average MAE': fold_results['MAE'].mean(),
        'Average MAPE': fold_results['MAPE'].mean(),
        'Average RMSE - MAE': fold_results['RMSE - MAE'].mean(),
        'Average RMSE - MAE %': fold_results['RMSE - MAE %'].mean()
    }

    # Output results
    print("\nFold-wise Performance:")
    print(fold_results)

    print("\nAverage Performance across all folds:")
    for k, v in avg_results.items():
        print(f"{k}: {v:.4f}")

    return best_model, fold_results, avg_results, X_test, y_test

#### __Plot Permutation Feature Importance for HGBoosted Model__

In [ ]:
def plot_permutation_importance(model, X_test, y_test, n_repeats=10, random_state=seed):
    """
    Computes and plots permutation feature importance.

    Parameters:
    - model: Trained model (already fitted).
    - X_test: Validation features (DataFrame) from last fold.
    - y_test: Validation target from last fold.
    - n_repeats: Number of times to permute each feature.
    - random_state: Seed for reproducibility.
    """

    result = permutation_importance(
        model, X_test, y_test,
        n_repeats=n_repeats,
        random_state=random_state,
        n_jobs=-1
    )

    importances = result.importances_mean
    feature_names = X_test.columns


    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    })

    # Separate positives and negatives
    pos_df = importance_df[importance_df['Importance'] > 0].sort_values(by='Importance', ascending=False)
    neg_df = importance_df[importance_df['Importance'] < 0].sort_values(by='Importance', ascending=True)
    neg_df = neg_df.iloc[::-1]  # Flip so negative values are increasingly negative.

    # Concatenate pos first, then neg
    importance_df_sorted = pd.concat([pos_df, neg_df])

    plt.figure(figsize=(10, max(6, len(feature_names) * 0.3)))

    # Plot all bars at once
    colors = ['tab:blue' if i > 0 else 'tab:orange' for i in importance_df_sorted['Importance']]
    plt.barh(importance_df_sorted['Feature'], importance_df_sorted['Importance'], color=colors)

    # Zero centered symmetric x-axis limits
    max_val = max(abs(importance_df_sorted['Importance'].min()), importance_df_sorted['Importance'].max())
    plt.xlim(-max_val * 1.1, max_val * 1.1)

    plt.xlabel('Mean Permutation Importance')
    plt.title('Permutation Feature Importance')
    plt.gca().invert_yaxis()  # Flips the y-axis so first row is at top
    plt.tight_layout()
    plt.show()

    print(importance_df_sorted)

#### __Get Max Temperature Reference Normals, Daily Mean Deviation and Standard Deviation__

In [ ]:
def get_daily_tmax_normals_mean_std(df, station_id='08221', k_neighbors=5):
    """
    Retrieve daily maximum temperature (tmax) reference normals and calculate
    historical deviation statistics for the city using Meteostat data (1991–2020).

    This function:
    - Downloads daily historical tmax data for the station.
    - Removes leap day (Feb 29) to standardize to a 365-day year.
    - Imputes missing values using KNN imputation.
    - Computes daily reference normals by averaging tmax across all years per day-of-year.
    - Calculates mean and standard deviation of daily deviations (actual - normal).

    Parameters:
    - station_id (str): Meteostat station ID (default: _____)
    - k_neighbors (int): Number of neighbors for KNN imputation (default: 5)

    Returns:
    - df (pd.DataFrame): DataFrame with a DatetimeIndex to determine the time window.
    - ref_tmax_dict: dict mapping day_of_year (1–365) → average tmax
    - mean_deviation: float, mean of all daily temperature deviations
    - std_deviation: float, std dev of all daily temperature deviations
    """
    # Fetch data from the area. The airport is considered to be
    # a good representation of weather data for the Community.

    # Check DataFrame has datetime index.
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("Input df must have a DatetimeIndex.")

    # Derive dynamic time window for data collection.
    end_date = df.index.min()
    start_date = end_date - pd.DateOffset(years=30)
    print(f"Fetching normals from {start_date.date()} to {end_date.date()}")

    # Fetch data from Meteostat.
    print("Fetching data...")
    data = Daily(station_id, start_date, end_date).fetch()
    data = data[['tmax']].copy()

    # Impute missing values using KNN.
    if data['tmax'].isna().sum() > 0:
        print(f"Missing values before imputation: {data['tmax'].isna().sum()}")
        imputer = KNNImputer(n_neighbors=k_neighbors)
        data[['tmax']] = imputer.fit_transform(data[['tmax']])
        print("Missing values imputed.")

    # Select data which is NOT February 29. This will leave gaps for leap years
    # which will need to be imputed in the dataset.
    data = data[~((data.index.month == 2) & (data.index.day == 29))]

    # Add day of year.
    data['day_of_year'] = data.index.dayofyear

    # Calculate daily normals
    print("Calculating daily normals...")
    daily_normals = data.groupby('day_of_year')['tmax'].mean().reindex(range(1, 366))
    daily_normals.name = 'ref_tmax'

    # Merge normals back for deviation calculation.
    data = data.merge(daily_normals, left_on='day_of_year', right_index=True)
    data['temp_deviation'] = data['tmax'] - data['ref_tmax']

    # Calculate statistics.
    mean_deviation = data['temp_deviation'].mean()
    std_deviation = data['temp_deviation'].std()

    print(f"Mean deviation: {mean_deviation:.2f}, Std deviation: {std_deviation:.2f}")

    return daily_normals.round(2).to_dict(), mean_deviation, std_deviation

#### __Prepare Forecasted Unit Sales with z-scores for Temperature Deviation from Normal__

In [ ]:
def prepare_forecast_with_z_scores(forecast_df, ref_normals, mean_deviation, std_deviation, k_neighbors=5):
    """
    Enhance a daily temperature forecast DataFrame by calculating deviation
    from historical reference normals and computing z-scores and confidence
    weights based on historical variability.

    This function:
    - Maps each forecasted date to its day-of-year (1–365).
    - Joins reference normals (1991–2020 average tmax) based on day-of-year.
    - Uses KNN imputation to fill in missing reference normals (e.g., Feb 29).
    - Calculates temperature deviation: forecast_tmax - ref_tmax.
    - Computes a z-score using provided historical mean and standard deviation
      of deviation from normal.
    - Assigns a confidence weighting to each forecast based on z-score thresholds
      (lower z = higher confidence).

    Parameters:
    - forecast_df (pd.DataFrame): Must include 'date' (datetime64) and 'forecast_tmax' columns.
    - ref_normals (dict): Mapping from day_of_year (1–365) to reference tmax.
    - mean_deviation (float): Historical mean deviation from normal.
    - std_deviation (float): Historical standard deviation of deviation.
    - k_neighbors (int, optional): Number of neighbors to use for KNN imputation (default: 5).

    Returns:
    - pd.DataFrame: Original forecast_df with the following added columns:
        - 'day_of_year': Day of year (1–365)
        - 'ref_tmax': Reference normal temperature
        - 'temp_deviation': Forecast - normal
        - 'z_score': Standardized deviation
        - 'confidence_weight': Heuristic weighting. A value between 0 (no confidence)
           and 1 (very confident), based on the z-score
    """
    df = forecast_df.copy()

    # Ensure the index is datetime
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("DataFrame must have a DatetimeIndex.")

    # Generate the day of year for reference mapping.
    df['day_of_year'] = df.index.dayofyear

    # Map reference normals.
    df['ref_tmax'] = df['day_of_year'].map(ref_normals)

    # Impute any missing ref_tmax (e.g., Feb 29) using KNN.
    if df['ref_tmax'].isna().sum() > 0:
        imputer = KNNImputer(n_neighbors=k_neighbors)
        df[['ref_tmax']] = imputer.fit_transform(df[['ref_tmax']])

    # Calculate deviation from reference normal.
    df['temp_deviation'] = df['forecasted_max_temp'] - df['ref_tmax']

    # Calculate the Z-score (how far the deviation is in std devs).
    df['z_score'] = (df['temp_deviation'] - mean_deviation) / std_deviation

    # Confidence weighting. Corresponds to a roughly normal distribution where
    # approx. 68% values lie within 1 std dev, 95% within 2 and 99% within 3.
    # Can be adjusted to be more aggressive.
    def confidence_weight(z):
        z = abs(z)
        if z < 1:
            return 0.7
        elif z < 2:
            return 0.5
        elif z < 3:
            return 0.3
        else:
            return 0.2

    df['confidence_weight'] = df['z_score'].apply(confidence_weight)

    return df

#### __Adjust Ratios Based on Confidence Weighting and Temperature Thresholds and Calculate Revenue__

In [ ]:
def adjust_ratios_calc_revenue(df, base_unit2_ratio=0.02, max_unit2_ratio=0.10,
                               unit2_price=1.89, unit_price=1.49,
                               unit2_margin=0.4, unit_margin=0.2,
                               prob_threshold=0.5, temp_threshold=28):
    """
    Adjust premium (unit2) vs standard (unit) product ratio based on:
      - Probability day exceeds 28°C (prob_exceed_28)
      - Conditional mean temperature (conditional_mean_temp)
      - Forecast uncertainty (z_score)
      - Event days affecting unit demand

    Parameters:
    - df (pd.DataFrame): Must have columns:
        'prob_exceed_28', 'conditional_mean_temp', 'z_score', 'is_event_day',
        'up_80', 'up_95'
    - base_unit2_ratio: Default premium product ratio when cool
    - max_unit2_ratio: Max premium ratio for hot days
    - unit2_price, unit_price, unit2_margin, unit_margin: Pricing and margins
    - prob_threshold: Probability cutoff to start adjusting above base ratio
    - temp_threshold: Temperature threshold for boosting signal

    Returns:
    - df with adjusted ratios, revenue and profit columns added
    """

    df = df.copy()

    # Vectorized selection of sold unit CI band based on whether an event occurs or not.
    df['total_units'] = np.where(df['is_event_day'], df['up_95'], df['up_80'])

    # Compute base heat signal from probability (scaled 0 to 1).
    df['heat_signal'] = np.clip((df['prob_exceed_28'] - prob_threshold) / (1 - prob_threshold), 0, 1)

    # Temperature boost.
    df['temp_boost'] = np.clip((df['conditional_mean_temp'] - temp_threshold) / 10, 0, 1)
    df['temp_boost'] = df['temp_boost'].fillna(1.0)

    # Combine heat signal with temp boost.
    df['heat_signal'] *= df['temp_boost']

    # Dampening for high uncertainty forecasts.
    df.loc[df['z_score'].abs() > 3, 'heat_signal'] *= 0.7

    # Below-threshold probability gets no heat signal, keeps base ratio of 2%.
    df.loc[df['prob_exceed_28'] < prob_threshold, 'heat_signal'] = 0

    # Adjusted ratios.
    df['adjusted_unit2_ratio'] = base_unit2_ratio + (max_unit2_ratio - base_unit2_ratio) * df['heat_signal']
    df['adjusted_unit2_ratio'] = df['adjusted_unit2_ratio'].clip(lower=base_unit2_ratio, upper=max_unit2_ratio)
    df['adjusted_unit_ratio'] = 1 - df['adjusted_unit2_ratio']

    # Revenue and profit calculations.
    df['unit2_revenue'] = df['adjusted_unit2_ratio'] * df['total_units'] * unit2_price
    df['unit_revenue'] = df['adjusted_unit_ratio'] * df['total_units'] * unit_price
    df['total_revenue'] = df['unit2_revenue'] + df['unit_revenue']
    df['expected_profit'] = (
        df['adjusted_unit2_ratio'] * df['total_units'] * unit2_price * unit2_margin +
        df['adjusted_unit_ratio'] * df['total_units'] * unit_price * unit_margin
    )

    return df

## 🔹 Target and Predictor Analysis

#### __Import and Validate filtered_sales_temp_holidays_and_matches.csv Dataset for Modelling__

In [ ]:
# Locate the dataset in Google Drive.
filepath_sales_events_temp = '/content/drive/MyDrive/Employer Project/Datasets/Production Data/filtered_sales_temp_hols_fmatches/final_events_clean.csv'

# Load the dataset.
sales_events_temp = pd.read_csv(filepath_sales_events_temp)

In [ ]:
# View dataframe.
sales_events_temp

In [ ]:
# Determine metadata and accurate import.
sales_events_temp.info()

#### __Address missing rows for 0 sales per sales location__
In the sales_events_temp dataset, each sales_location represents a unique location, and sold_units reflects daily sales. We identified that some sales_locations lack records for certain dates—not due to missing data, but because no sales occurred.

To create a consistent time series, we assume that missing rows indicate zero sales. We generated all combinations of sales_location and date, filling in missing rows with sold_units = 0. Other fields in these rows are populated based on existing entries for the same date to maintain consistency.

In [ ]:
# Change date column to datetime format
sales_events_temp['date'] = pd.to_datetime(sales_events_temp['date'])

# Generate all dates in 2021 and 2022
all_dates = pd.date_range(start='2021-01-01', end='2022-12-31')

# Extract unique dates for 2021 and 2022
existing_dates = sales_events_temp[
    sales_events_temp['date'].dt.year.isin([2021, 2022])
]['date'].dt.normalize().unique()

# Convert to datetime index for comparison
existing_dates = pd.to_datetime(existing_dates)

# Find missing dates
missing_dates = all_dates.difference(existing_dates)

# Display the result
print("Missing dates in 2022:")
print(missing_dates)

In [ ]:
# Define all valid (non-missing) dates in 2022
all_dates = pd.date_range(start='2021-01-01', end='2022-12-31')
existing_dates = pd.to_datetime(
    sales_events_temp[
        sales_events_temp['date'].dt.year.isin([2021, 2022])
    ]['date'].dt.normalize().unique()
)

missing_dates = all_dates.difference(existing_dates)
valid_dates = pd.DatetimeIndex(existing_dates)

# Get all unique sales station codes
stations = sales_events_temp['sales_location'].unique()

# Create full grid of station × date
full_grid = pd.MultiIndex.from_product(
    [stations, valid_dates],
    names=['sales_location', 'date']
).to_frame(index=False)

# Merge with actual 2022 data
existing_data = sales_events_temp[sales_events_temp['date'].dt.year.isin([2021, 2022])]
merged = full_grid.merge(
    existing_data,
    on=['sales_location', 'date'],
    how='left')

# Fill missing `sold_units` with 0
merged['sold_units'] = merged['sold_units'].fillna(0)

In [ ]:
# Define columns that should be filled based on 'date'
date_based_columns = [
    'day', 'month', 'year', 'month_text', 'day_of_week', 'season',
    'avg_temp', 'min_temp', 'max_temp', 'precip', 'wind_dir',
    'wind_speed', 'avg_pressure', 'holiday_name',
    'sport_match', 'sport1_match',
    'is_holiday', 'is_sport', 'is_sport1', 'is_match', 'event_category'
]

# Fill missing values by mapping values from other rows with the same date
# Step 1: Create a lookup table with one row per date
date_lookup = merged[~merged[date_based_columns].isna().all(axis=1)].groupby('date')[date_based_columns].first()

# Merge this back to the original DataFrame on 'date' to fill missing values
merged_filled = merged.drop(columns=date_based_columns).merge(
    date_lookup, how='left', on='date'
)

# Done — check if missing values are resolved
missing_after = merged_filled[date_based_columns].isna().sum()
print("Missing values after filling based on date:")
print(missing_after)

In [ ]:
# List of columns to fill based on sales_location
station_info_cols = ['province', 'town/city', 'postcode', 'company_code', 'sales_location']

# Fill missing values by group (based on sales_location)
merged_filled[station_info_cols] = (
    merged_filled
    .groupby('sales_location')[station_info_cols]
    .transform(lambda group: group.ffill().bfill())
)
print(merged_filled[station_info_cols].isna().sum())

In [ ]:
# Convert columns in merged_filled to match the data types of sales_events_temp

# Adjust int64 columns (e.g., company_code, sales_location, etc.)
int_columns = ['company_code', 'postcode', 'sales_location', 'day', 'month', 'year', 'sold_units']  # example of int columns
merged_filled[int_columns] = merged_filled[int_columns].astype('int64')

# Adjust float64 columns (e.g., avg_temp, min_temp, max_temp, etc.)
float_columns = ['avg_temp', 'min_temp', 'max_temp', 'precip', 'wind_dir', 'wind_speed', 'avg_pressure']
merged_filled[float_columns] = merged_filled[float_columns].astype('float64')

# Adjust object columns (e.g., province, town/city, etc.)
object_columns = ['province', 'town/city', 'sales_location', 'month_text', 'day_of_week', 'season', 'holiday_name', 'event_category']
merged_filled[object_columns] = merged_filled[object_columns].astype('object')

# Adjust boolean columns
boolean_columns = ['is_holiday', 'is_sport', 'is_sport1', 'is_match']
merged_filled[boolean_columns] = merged_filled[boolean_columns].astype('bool')

# Check the updated data types in merged_filled
sales_events_temp = merged_filled
sales_events_temp.info()

#### __Address Missing Weather Variables__

In [ ]:
# Fill missing avg_temp values.
sales_events_temp['avg_temp'] = sales_events_temp['avg_temp'].fillna(
    (
        sales_events_temp['max_temp'] + sales_events_temp['min_temp']
    ) / 2
)

Average temperature calculated as the maximum + minimum / 2 is a commonly used method to fill missing average temperatures. To ensure consistency with how missing average temperatures are typically resolved, this technique has been utilised here.

In [ ]:
# Select columns which require filling.
cols = ['wind_dir', 'wind_speed', 'avg_pressure']
original_data = sales_events_temp[cols]

# Perform KNN imputation.
imputer = KNNImputer(n_neighbors=5)  # Use k=5 for KNN imputation.
imputed_weather_values = pd.DataFrame(imputer.fit_transform(original_data), columns=cols, index=original_data.index)

# Round imputed DataFrame.
imputed_weather_values = imputed_weather_values.round(1)

# Use fillna to only update missing values.
for col in cols:
    sales_events_temp[col] = original_data[col].fillna(imputed_weather_values[col])

KNN imputation is commonly used to fill missing weather data. It leverages the similarities and temporal nature of weather events to impute the missing values within the dataset and is generally found to be more accurate than mean or median. A k of 5 is chosen as the missing values comprise approximately 0.5% of the dataset and a k of five is assumed to roughly capture the short-term weather patterns and gradual changes.

#### __Geocode Postcodes to Generate Latitude and Longitude Values for Modelling__

In [ ]:
# Check for unique postcodes in the dataset.
postcodes_coord = pd.DataFrame(sales_events_temp['postcode'].drop_duplicates())

postcodes_coord

In [ ]:
# Retrieve the lat and lon of for each postcode.
postcodes_coord[['latitude', 'longitude']] = postcodes_coord['postcode'].apply(
    lambda x: pd.Series(get_coordinates_from_postcode(x))
)

In [ ]:
# Sense-check coordinates against postcodes.
postcodes_coord

In [ ]:
# Check for null coordinate values.
postcodes_coord.info()

In [ ]:
# Create a dictionary: {postcode: (lat, lon)} from postcodes_coord DataFrame.
# Set 'postal_code' as the index.
postcodes_coord = postcodes_coord.set_index('postcode')

# Select latitude and longitude columns and convert to a tuple.
postcodes_coord = postcodes_coord[['latitude', 'longitude']].apply(tuple, axis=1)

# Convert the Series to a dictionary.
postcode_coords_dict = postcodes_coord.to_dict()

In [ ]:
# Copy the sales_events_temp DataFrame before geocoding.
sales_events_temp_geo = sales_events_temp.copy()

# Map lat and lon to sales_events_temp DataFrame.
sales_events_temp_geo[['latitude', 'longitude']] = sales_events_temp_geo['postcode'].map(postcode_coords_dict).apply(pd.Series)

sales_events_temp_geo.info()

In [ ]:
# Define new column order following logical grouping.
column_order = [
    "province", "town/city", "postcode", "latitude", "longitude",
    "company_code", "sales_location", "sales_location",
    "date", "year", "month", "month_text", "day", "day_of_week", "season",
    "sold_units",
    "avg_temp", "min_temp", "max_temp", "precip",
    "wind_dir", "wind_speed", "avg_pressure",
    "holiday_name", "is_holiday",
    "sport_match", "is_sport", "sport1_match", "is_sport1", "is_match"
]

# Reorder DataFrame columns.
sales_events_temp_geo = sales_events_temp_geo[column_order]

# View DataFrame.
sales_events_temp_geo.info()

In [ ]:
# Download the geocoded dataset as a CSV.
sales_events_temp_geo.to_csv('/content/drive/MyDrive/Employer Project/Datasets/Production Data/filtered_sales_temp_hols_fmatches/sales_events_temp_geocoded.csv', index=False)

#### __Import and Validate Max Temp Reference Normals__

##### __Use Meteostat to Generate and Store Daily Rolling Reference for the metropolitan area 2021 and 2022__

In [ ]:
# Ensure date column is datetime.
sales_events_temp_geo['date'] = pd.to_datetime(sales_events_temp_geo['date'])

# Create a day-of-year column (1 to 365).
sales_events_temp_geo['day_of_year'] = sales_events_temp_geo['date'].dt.dayofyear

In [ ]:
# Retrieve reference normal values for 2021 and 2022.
normals_2021 = get_daily_temp_normals(start_year=2021, end_year=2021)
normals_2022 = get_daily_temp_normals(start_year=2022, end_year=2022)

# Filter the dataset for 2021 and 2022.
sales_events_2021 = sales_events_temp_geo[sales_events_temp_geo['year'] == 2021]
sales_events_2022 = sales_events_temp_geo[sales_events_temp_geo['year'] == 2022]

# Map normals for 2021.
sales_events_2021['ref_tmax'] = sales_events_2021['day_of_year'].map(normals_2021['ref_tmax'])
sales_events_2021['ref_tmin'] = sales_events_2021['day_of_year'].map(normals_2021['ref_tmin'])
sales_events_2021['ref_tavg'] = sales_events_2021['day_of_year'].map(normals_2021['ref_tavg'])

# Map normals for 2022.
sales_events_2022['ref_tmax'] = sales_events_2022['day_of_year'].map(normals_2022['ref_tmax'])
sales_events_2022['ref_tmin'] = sales_events_2022['day_of_year'].map(normals_2022['ref_tmin'])
sales_events_2022['ref_tavg'] = sales_events_2022['day_of_year'].map(normals_2022['ref_tavg'])

# Combine both years back together.
sales_events_temp_geo = pd.concat([sales_events_2021, sales_events_2022])

# Check if any missing values exist.
if sales_events_temp_geo[['ref_tmax', 'ref_tmin', 'ref_tavg']].isna().sum().sum() > 0:
    print(f"Missing values detected, applying KNN imputation...")

    # Prepare for imputation.
    for var in ['ref_tmax', 'ref_tmin', 'ref_tavg']:
        if sales_events_temp_geo[var].isna().sum() > 0:
            original_values = sales_events_temp_geo[[var]].copy()

            # Perform KNN imputation
            imputer = KNNImputer(n_neighbors=5)
            imputed_values = pd.DataFrame(imputer.fit_transform(original_values),
                                          columns=[var],
                                          index=original_values.index)

            # Round and update missing values
            sales_events_temp_geo[var] = original_values[var].fillna(imputed_values[var].round(2))

    print("Missing values imputed successfully.")
else:
    print("No missing values found.")

# Sense check the DataFrame
sales_events_temp_geo

In [ ]:
# Look for missing reference temperatures.
sales_events_temp_geo.info()

##### __Finalise DataFrame for Initial Visualisation__

In [ ]:
# Drop uneccessary columns for future modelling.
sales_hols_reftemp_geo = sales_events_temp_geo.drop(
    columns=[
        'holiday_name',
        'sport_match',
        'sport1_match'
    ]
)

sales_hols_reftemp_geo.info()

In [ ]:
# Define new column order following logical grouping.
column_order = [
    "province", "town/city", "postcode", "latitude", "longitude",
    "company_code", "sales_location", "sales_location",
    "date", "year", "month", "month_text", "day", "day_of_year", "day_of_week",
    "season", "sold_units", "avg_temp", "min_temp", "max_temp",
    "ref_tavg", "ref_tmin", "ref_tmax", "precip", "wind_dir", "wind_speed",
    "avg_pressure", "is_holiday", "is_sport", "is_sport1", "is_match"
]

# Reorder DataFrame columns.
sales_hols_reftemp_geo = sales_hols_reftemp_geo[column_order]

# View DataFrame.
sales_hols_reftemp_geo.info()

In [ ]:
# Download the geocoded dataset as a CSV.
sales_hols_reftemp_geo.to_csv('/content/drive/MyDrive/Employer Project/Datasets/Production Data/filtered_sales_temp_hols_fmatches/sales_hols_reftemp_geocoded.csv', index=False)

#### __Variable Correlation__

In [ ]:
# Ensure that the appropriate variables are categorical vs. numerical.
# Convert postcode to string to preserve any leading zeros or alphanumeric codes.
sales_hols_reftemp_geo['postcode'] = sales_hols_reftemp_geo['postcode'].astype(str)

# Convert date to datetime format.
sales_hols_reftemp_geo['date'] = pd.to_datetime(sales_hols_reftemp_geo['date'])

# Convert company_code and sales_location to string (identifiers should be stored as strings).
sales_hols_reftemp_geo['company_code'] = sales_hols_reftemp_geo['company_code'].astype(str)
sales_hols_reftemp_geo['sales_location'] = sales_hols_reftemp_geo['sales_location'].astype(str)

# Verify the changes by checking the metadata.
sales_hols_reftemp_geo.info()

In [ ]:
# Check high-level correlation between variables.
corr_matrix = sales_hols_reftemp_geo.corr(numeric_only=True)

# Create the heatmap for correlation
plt.figure(figsize=(16, 16))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap")
plt.show()

There are many limitations to this correlation plot, we know this is largely looking at linear correlations, and we can see the temperature/climate variables demonstrate stronger correlation than other variables which were demonstrated to at least visually have a relationship to the target variable. This plot should therefore be view skeptically. It does not necessarily demonstrate the complexity of the relationships seen visually. There is likely a lot of colinearity between between climate, holidays, and other temporal variables. High heat occurs on certain days of the year, seasons and months and there is even a temporal relationship to sports events which also occurs cyclically. It is very doubtful that these complex relationships could be captured accurately by a simpler model such as multiple linear regression or even ARIMA.

##### __Refine Variables__

In [ ]:
# Clean up redundant variables. Some variables will be removed based on
# performance of models.
sales_modelling_variables = sales_hols_reftemp_geo.drop(columns=['postcode',
                                                                 'month_text',
                                                                 ])

sales_modelling_variables.info()

##### __Encode Required Variables__

In [ ]:
# Ensure order for season variable.
season_order = ['Winter', 'Spring', 'Summer', 'Autumn']

sales_modelling_variables['season'] = pd.Categorical(sales_modelling_variables['season'], categories=season_order, ordered=True)

# One-hot encode required variables. None have inherent ordinality.
sales_modelling_vars_one_hot = one_hot_encode_columns(sales_modelling_variables, ['season'], prefix=True, drop_first=True, keep_original=True)

# Sense check DataFrame.
sales_modelling_vars_one_hot

In [ ]:
# Ensure cyclical variables are ordered and numeric.
# Ensure order for categorical day of week variable.
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sales_modelling_vars_one_hot['day_of_week'] = pd.Categorical(sales_modelling_vars_one_hot['day_of_week'], categories=day_order, ordered=True)


# List of cyclical variables and their expected index base (0 or 1)
cyclical_variables_info = {
    'month': {'period': 12, 'index_base': 1},
    'day_of_week': {'period': 7, 'index_base': 0},
    'day_of_year': {'period': 365, 'index_base': 1},
}

# Make a copy of the DataFrame to work safely.
sales_modelling_variables_encoded = sales_modelling_vars_one_hot.copy()

for col, info in cyclical_variables_info.items():
    # Convert categorical or object columns to codes.
    if sales_modelling_variables_encoded[col].dtype.name in ['category', 'object']:
        sales_modelling_variables_encoded[col] = pd.Categorical(sales_modelling_variables_encoded[col]).codes

    # Force numeric, coerce errors to NaN for safety.
    sales_modelling_variables_encoded[col] = pd.to_numeric(sales_modelling_variables_encoded[col], errors='coerce')

    # Ensure 0-based or 1-based indexing as specified.
    if info['index_base'] == 1 and sales_modelling_variables_encoded[col].min() == 0:
        sales_modelling_variables_encoded[col] = sales_modelling_variables_encoded[col] + 1  # Shift to 1-based where required.

# Sense check the DataFrame.
sales_modelling_variables_encoded

In [ ]:
# Define periods for cyclical encoding.
cyclical_columns = {
    'month': 12,
    'day_of_week': 7,
    'day_of_year': 365
}

# Apply cyclical encoding.
sales_modelling_encoded = add_cyclical_encoding(sales_modelling_variables_encoded, columns_periods=cyclical_columns, drop_original=False)

# View dataframe.
sales_modelling_encoded

In [ ]:
# Sense check DataFrame.
sales_modelling_encoded.info()

In [ ]:
# Set logical order for variables.
column_order = [
    'province',
    'town/city',
    'latitude',
    'longitude',
    'company_code',
    'sales_location',
    'sold_units',
    'date',
    'year',
    'month',
    'month_sin',
    'month_cos',
    'day',
    'day_of_week',
    'day_of_week_sin',
    'day_of_week_cos',
    'day_of_year',
    'day_of_year_sin',
    'day_of_year_cos',
    'season_Spring',
    'season_Summer',
    'season_Autumn',
    'avg_temp',
    'min_temp',
    'max_temp',
    'ref_tavg',
    'ref_tmin',
    'ref_tmax',
    'precip',
    'wind_dir',
    'wind_speed',
    'avg_pressure',
    'is_holiday',
    'is_sport',
    'is_sport1',
    'is_match'
]

# Reorder DataFrame columns.
sales_modelling_encoded  = sales_modelling_encoded [column_order]

# View DataFrame.
sales_modelling_encoded.info()

In [ ]:
# Save to csv.
sales_modelling_encoded.to_csv('/content/drive/MyDrive/Employer Project/Datasets/Production Data/filtered_sales_temp_hols_fmatches/sales_modelling_encoded.csv', index=False)

##### __Revisit Correlation with Encoded Variables__

In [ ]:
# Subset sales_modelling_encoded for correlation visulisation.
sales_modelling_encoded_sub = sales_modelling_encoded[[
    'latitude', 'longitude', 'sold_units', 'month_sin', 'month_cos',
    'day_of_week_sin', 'day_of_week_cos', 'day_of_year_sin', 'day_of_year_cos',
    'season_Spring', 'season_Summer', 'season_Autumn',
    'avg_temp', 'min_temp', 'max_temp', 'precip',
    'wind_dir', 'wind_speed', 'avg_pressure', 'is_holiday', 'is_sport',
    'is_sport1'
]]

# Check high-level correlation between variables.
corr_matrix = sales_modelling_encoded_sub.corr(numeric_only=True)

# Create the heatmap for correlation
plt.figure(figsize=(16, 16))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap")
plt.show()

Encoding temporal variables cyclically has improved the visible correlation for these variables. This demonstrates that perhaps the relationship between these temporal variables and the sold units is best represented this way, capturing the true cyclical nature of time (which will be important for the model), but also that again the relationships are unlikely to be modelled well with a simpler model that fails to capture the complex and intertwined nature of all the predictor variables. As we can see some of these relationships established here.

In [ ]:
# Convert boolean columns to integers.
bool_cols = sales_modelling_encoded_sub.select_dtypes(include='bool').columns
sales_modelling_encoded_sub[bool_cols] = sales_modelling_encoded_sub[bool_cols].astype(int)

# Calculate the VIF for each variable.
vif_data = pd.DataFrame()
features = sales_modelling_encoded_sub.drop('sold_units', axis=1)
vif_data["feature"] = features.columns
vif_data["VIF"] = [
    variance_inflation_factor(features.values, i)
    for i in range(len(features.columns))
]

vif_data

In [ ]:
# Drop potentially redundant columns.
sales_modelling_encoded_vif = sales_modelling_encoded_sub[[
    'latitude', 'longitude', 'month_sin', 'month_cos',
    'day_of_week_sin', 'day_of_week_cos', 'day_of_year_sin', 'day_of_year_cos',
    'season_Spring', 'season_Summer', 'season_Autumn',
    'max_temp', 'precip', 'wind_dir', 'wind_speed', 'is_holiday', 'is_sport',
    'is_sport1'
]]

# Create a DataFrame to capture VIF values.
vif_data = pd.DataFrame()
vif_data["feature"] = sales_modelling_encoded_vif.columns

# Convert boolean columns to integers.
bool_cols = sales_modelling_encoded_vif.select_dtypes(include='bool').columns
sales_modelling_encoded_vif[bool_cols] = sales_modelling_encoded_vif[bool_cols].astype(int)

# Calculate the VIF for each variable.
vif_data = pd.DataFrame()
vif_data["feature"] = sales_modelling_encoded_vif.columns
vif_data["VIF"] = [
    variance_inflation_factor(sales_modelling_encoded_vif.values, i)
    for i in range(len(sales_modelling_encoded_vif.columns))
]

vif_data

In [ ]:
sales_model_corr_vif = sales_modelling_encoded[[
    'latitude', 'longitude', 'sold_units', 'month_sin', 'month_cos',
    'day_of_week_sin', 'day_of_week_cos', 'day_of_year_sin', 'day_of_year_cos',
    'season_Spring', 'season_Summer', 'season_Autumn',
    'max_temp', 'precip', 'wind_dir', 'wind_speed', 'is_holiday', 'is_sport',
    'is_sport1'
]]

# Check high-level correlation between variables.
corr_matrix = sales_model_corr_vif.corr(numeric_only=True)

# Create the heatmap for correlation
plt.figure(figsize=(16, 16))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap")
plt.show()

While colinearity is not as much of an issue for more complex models, looking at the VIF demonstrates well again the complex nature of these relationships. It may be interesting to establish the linear relationship between sold units and temperature in order to establish possibly 'lost sales' but is unlikely to be worth any further investigation.

In [ ]:
# Refine final DataFrame for modelling.
sales_model_final = sales_modelling_encoded[[
    'province',
    'town/city',
    'latitude',
    'longitude',
    'company_code',
    'sales_location',
    'sold_units',
    'date',
    'year',
    'month',
    'month_sin',
    'month_cos',
    'day',
    'day_of_week',
    'day_of_week_sin',
    'day_of_week_cos',
    'day_of_year',
    'day_of_year_sin',
    'day_of_year_cos',
    'season_Spring',
    'season_Summer',
    'season_Autumn',
    'avg_temp',
    'min_temp',
    'max_temp',
    'ref_tavg',
    'ref_tmin',
    'ref_tmax',
    'precip',
    'wind_dir',
    'wind_speed',
    'avg_pressure',
    'is_holiday',
    'is_sport',
    'is_sport1'
]]

sales_model_final.info()

In [ ]:
# Save the final modelling DataFrame.
sales_model_final.to_csv('/content/drive/MyDrive/Employer Project/Datasets/Production Data/filtered_sales_temp_hols_fmatches/sales_model_final.csv', index=False)

# 🔹 HistGradientBoostingRegressor

The following features were selected because the model inherently captures time-based patterns, such as weekly, monthly, and seasonal trends, through inputs like day of year and sine/cosine transformations of weekday and month.


In [ ]:
df_hgb = sales_model_final.copy()[[
    'sales_location',
    'sold_units',
    'date',
    'day_of_week_cos',
    'day_of_week_sin',
    'year',
    'month',
    'month_sin',
    'month_cos',
    'day',
    'day_of_week',
    'day_of_year',
    'avg_temp',
    'min_temp',
    'max_temp',
    'precip',
    'wind_dir',
    'wind_speed',
    'avg_pressure',
    'is_holiday',
    'town/city',
    'company_code',
    'province'
]]

In [ ]:
# Create named df & sort by date to prepare for timeseriessplit
df_hgb['date'] = pd.to_datetime(df_hgb['date'])
df_hgb = df_hgb.sort_values(by='date')

df_hgb['week'] = df_hgb['date'].dt.isocalendar().week

In [ ]:
df_hgb.head()

Since many sales locations sold zero units on over half the days in 2021 and 2022, we aggregated sold units at the town/city level to help the model recognize patterns and improve predictability.

In [ ]:
# 1. Aggregate data by 'date' and 'town/city'
df_hgb['date'] = pd.to_datetime(df_hgb['date'])

df_agg = df_hgb.groupby(['date', 'town/city']).agg({
    'sold_units': 'sum',
    'avg_temp': 'mean',
    'min_temp': 'mean',
    'max_temp': 'mean',
    'precip': 'mean',
    'wind_dir': 'mean',
    'wind_speed': 'mean',
    'avg_pressure': 'mean',
    'is_holiday': 'max',  # assuming binary
    'province': 'first',
    'company_code': 'first',
    'week': 'first',
    'day_of_week_cos': 'mean',
    'day_of_week_sin': 'mean',
    'year': 'first',
    'month': 'first',
    'month_sin': 'mean',
    'month_cos': 'mean',
    'day': 'first',
    'day_of_week': 'first',
    'day_of_year': 'first'
}).reset_index()

# 2. Encode categorical columns ('town/city', 'province', 'company_code')
label_encoders = {}

for col in ['town/city', 'province', 'company_code']:
    le = LabelEncoder()
    df_agg[col] = le.fit_transform(df_agg[col])
    label_encoders[col] = le

# 3. Sort by date and set date index for timeseries splitting
df_agg = df_agg.sort_values(by='date')
df_agg.set_index('date', inplace=True)

# 4. Prepare features and target for modeling
X = df_agg.drop(columns=['sold_units'])
y = df_agg['sold_units']

In [ ]:
# Evaluate time series splits
evaluate_time_series_split_masked(HistGradientBoostingRegressor(), df_agg, y_col='sold_units')

We decided to proceed with 9 splits, as it offers the lowest mean MSE (333.99) among all configurations, indicating the best overall predictive accuracy. Additionally, it maintains a relatively low standard deviation (214.68), suggesting consistent performance across folds. This setup strikes a strong balance between evaluation stability and model performance without excessively fragmenting the time series.

In [ ]:
# 5. Create TimeSeriesSplit (assuming you have your custom function)
splits = get_timeseries_splits_with_masking(df_agg, 'sold_units', n_splits=9)

The hyperparameters were selected as follows to ensure optimal model performance.

In [ ]:
# Choose hyperparameters
params = {
    'max_leaf_nodes': 40,
    'min_samples_leaf': 25,
    'l2_regularization': 1.0,
    'random_state': seed
}

# Tune model with chosen parameters.
final_model, fold_rmse, cv_rmse = tune_hgb_model_with_masking(
    df=df_agg,
    y='sold_units',
    params=params,
    n_splits=9,
    random_state=42
)

In [ ]:
# Initialize lists to store performance metrics for each fold
best_model, fold_results, avg_results, X_test, y_test = evaluate_hgb_performance_with_best_model_masked(
    df_agg, 'sold_units', n_splits=9, best_model=final_model
)

As a next step, we are removing features with negative permutation importance, as these features reduce model performance by introducing noise rather than contributing useful predictive information.

In [ ]:
# Calculate permutation importance with explicit scoring (e.g., neg_mean_squared_error for regression)
result = permutation_importance(
    best_model, X_test, y_test,
    n_repeats=10, random_state=seed,
    scoring='neg_mean_squared_error'
)

# Convert to pandas Series
importance_scores = pd.Series(result.importances_mean, index=X_test.columns)

# Print all feature importances sorted
print("Permutation feature importances (mean decrease in score):")
print(importance_scores.sort_values(ascending=False))

# Identify negative features (features that *improve* the score when permuted)
negative_features = importance_scores[importance_scores < 0].index.tolist()

print("\nNegative features based on permutation importance (possibly harming the model):")
print(negative_features)

In [ ]:
# Removing the negative features
df_agg = df_agg.drop(columns=negative_features)
df_agg.info()

As a next step, we created a file containing information about local events, such as sports events, which are somewhat predictable and typically recur annually around the same weeks or months. While it's not feasible to forecast the exact dates of these events in future years, we generated weekly and monthly event flags based on historical patterns. These features can be used in predictions to help the model account for the impact of recurring events.

In [ ]:
filepath_fixed_events_2 = '/content/drive/MyDrive/Employer Project/Datasets/Production Data/filtered_sales_temp_hols_fmatches/fixed_events_2.csv'

# Load the dataset.
events = pd.read_csv(filepath_fixed_events_2)

In [ ]:
events.head(10)

In [ ]:
# Ensure date columns are datetime
events['start_date'] = pd.to_datetime(events['start_date'])
events['end_date'] = pd.to_datetime(events['end_date'])

# STEP 1: Add 'event_day_flag' for specific event days
event_dates = set()
for _, row in events.iterrows():
    dates = pd.date_range(row['start_date'], row['end_date'])
    event_dates.update(dates)

# Add binary flag for each date
df_agg = df_agg.reset_index()
df_agg['event_day_flag'] = df_agg['date'].isin(event_dates).astype(int)

# STEP 2: Prepare weekly/monthly event counts from events
weekly_event_counts = events.groupby('weekly_flag')['weekly_event_count'].sum().reset_index()
monthly_event_counts = events.groupby('monthly_flag')['monthly_event_count'].sum().reset_index()

# STEP 3: Merge with main df using 'week' and 'month'
df_agg['week'] = df_agg['date'].dt.strftime('%Y-W%U')  # Match weekly_flag format
df_agg['month'] = df_agg['date'].dt.strftime('%Y-%m')  # Match monthly_flag format

df_agg = df_agg.merge(weekly_event_counts, how='left', left_on='week', right_on='weekly_flag')
df_agg = df_agg.merge(monthly_event_counts, how='left', left_on='month', right_on='monthly_flag')

# Fill missing event counts with 0 (i.e., no events that week/month)
df_agg['weekly_event_count'] = df_agg['weekly_event_count'].fillna(0)
df_agg['monthly_event_count'] = df_agg['monthly_event_count'].fillna(0)

# Final clean-up
df_agg = df_agg.drop(columns=['weekly_flag', 'monthly_flag'])

# Restore index
df_agg = df_agg.sort_values('date')
df_agg.set_index('date', inplace=True)

# After merging weekly/monthly counts
df_agg.drop(columns=['week', 'month'], inplace=True, errors='ignore')

df_agg = df_agg.drop(columns=['event_day_flag'])

X = df_agg.drop(columns=['sold_units'])
y = df_agg['sold_units']

In [ ]:
# Create TimeSeriesSplit.
splits = get_timeseries_splits_with_masking(df_agg, 'sold_units', n_splits=9)

In [ ]:
params = {
    'max_leaf_nodes': 40,
    'min_samples_leaf': 25,
    'l2_regularization': 1.0,
    'random_state': seed
}


final_model, fold_rmse, cv_rmse = tune_hgb_model_with_masking(
    df=df_agg,
    y='sold_units',
    params=params,
    n_splits=9,
    random_state=42
)

In [ ]:
# Initialize lists to store performance metrics for each fold
best_model, fold_results, avg_results, X_test, y_test = evaluate_hgb_performance_with_best_model_masked(
    df_agg, 'sold_units', n_splits=9, best_model=final_model
)

In [ ]:
# Check for overfitting
check_overfitting_on_last_fold_masked(df_agg, y_col='sold_units', n_splits=9, model=best_model)

In [ ]:
unique_dates = df_agg.index.unique().sort_values()
tscv = TimeSeriesSplit(n_splits=9)

for train_idx, test_idx in tscv.split(unique_dates):
    pass  # This iterates through all, so at the end you'll have the last indices

train_dates = set(unique_dates[train_idx])
test_dates = set(unique_dates[test_idx])

# Use df_agg's index to create masks, not hgb_model's
train_mask = df_agg.index.map(train_dates.__contains__)
test_mask = df_agg.index.map(test_dates.__contains__)

X = df_agg.drop(columns=['sold_units'])
y = df_agg['sold_units']

X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[test_mask], y[test_mask]

# Predict on training fold only
y_train_pred = best_model.predict(X_train)
train_residuals = y_train - y_train_pred

# 1. Residuals vs. Predicted (Train only)
plt.figure(figsize=(8, 5))
sns.scatterplot(x=y_train_pred, y=train_residuals, alpha=0.6)
plt.axhline(0, color='red', linestyle='--')
plt.title("Residuals vs. Predicted Values (Train Only)")
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.tight_layout()
plt.show()

# 2. Histogram of Residuals (Train only)
plt.figure(figsize=(8, 5))
sns.histplot(train_residuals, bins=50, kde=True)
plt.axvline(0, color='red', linestyle='--')
plt.title("Histogram of Residuals (Train Only)")
plt.xlabel("Residual")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()


In [ ]:
explainer = shap.Explainer(best_model)
shap_values = explainer(X_test)

shap.plots.beeswarm(shap_values)

In [ ]:
df_hgb = df_agg.copy()

### Observations
This final model demonstrates strong performance across the cross-validation folds. The average Mean Squared Error (MSE) is 287.50, with a Root Mean Squared Error (RMSE) of 16.05 and a Mean Absolute Error (MAE) of 6.38. The model explains about 66% of the variance in the target variable, as shown by the average R². A high Mean Absolute Percentage Error (MAPE) of 169.66% reflects considerable variability or scale issues in the target.

Overfitting is evident in the final fold, where training R² is 0.8917 and test R² drops to 0.6218, showing the model fits training data better than new data. Despite this, no further improvements will be made.

Residuals are mostly well distributed, with a slight leftward skew suggesting minor underprediction in some cases. Overall, no strong bias or heteroscedasticity is observed.

Incorporating more historical data could help reduce overfitting and lower the high MAPE by improving the model's generalization and handling of target variability. However, this model is considered final for the current project, and we will now proceed to build confidence intervals using Monte Carlo simulation to further assess its predictive uncertainty.

# 🔹 Prepare Features for Forecasting Period

## Check Model Features

In [ ]:
feature_names = best_model.feature_names_in_

feature_names

## Create Temporal Features

In [ ]:
# Create DataFrame with dates.
forecast_df_calendar = generate_static_calendar_features(2023)

forecast_df_calendar

## Repanel Forecast Dataframe

In [ ]:
X_test

In [ ]:
# Create forecast panel with location info
one_hot_cols = ['town/city', 'company_code']

# Remove station_code from one_hot_columns if present to avoid duplication
if 'town/city' == 'town/city' and 'town/city' in one_hot_cols:
    one_hot_cols.remove('town/city')

forecast_df_panel = create_forecast_panel(
    X_test,
    forecast_df_calendar,
    station_code='town/city',
    one_hot_columns=one_hot_cols
)

In [ ]:
for i, col in enumerate(forecast_df_panel.columns):
    print(i, repr(col))


## Add Holiday Flag

### Load Holidays Dataset

In [ ]:
# Load 2023 holidays data
filepath_city_holidays_2023 = '/content/drive/MyDrive/Employer Project/Datasets/Production Data/filtered_sales_temp_hols_fmatches/city_holidays_2023.csv'

# Load the dataset.
city_holidays_2023 = pd.read_csv(filepath_city_holidays_2023)

### Map Holiday Flag to Forecast Frame

In [ ]:
# Convert date to datetime ensruing the day first read
city_holidays_2023['date'] = pd.to_datetime(city_holidays_2023['date'], dayfirst=True)

# Create set of holiday dates
hols_2023 = set(city_holidays_2023['date'])

# Add is_holiday flag
forecast_df_panel['is_holiday'] = forecast_df_panel['date'].isin(hols_2023)

forecast_df_panel.info()

##Add event weekly and monthly flags

In [ ]:
events.info()

In [ ]:
# 1. Extract week and month from forecast_df_panel (for 2023)
forecast_df_panel['week'] = forecast_df_panel['date'].dt.isocalendar().week

# 2. Create average event counts per week and month from 2021–2022

# Make sure event columns are numeric
events['weekly_event_count'] = pd.to_numeric(events['weekly_event_count'], errors='coerce')
events['monthly_event_count'] = pd.to_numeric(events['monthly_event_count'], errors='coerce')

# Extract week and month from the event calendar
events['week'] = pd.to_datetime(events['start_date']).dt.isocalendar().week
events['month'] = pd.to_datetime(events['start_date']).dt.month

# Group by week and month to calculate average event counts (across 2021–2022)
weekly_avg = events.groupby('week')['weekly_event_count'].mean().reset_index()
monthly_avg = events.groupby('month')['monthly_event_count'].mean().reset_index()

# 3. Merge the averages into 2023 forecast panel
forecast_df_panel = forecast_df_panel.merge(weekly_avg, on='week', how='left')
forecast_df_panel = forecast_df_panel.merge(monthly_avg, on='month', how='left')

# 4. Rename the merged columns to match the model's expected feature names
forecast_df_panel.rename(columns={
    'weekly_event_count': 'weekly_event_count',
    'monthly_event_count': 'monthly_event_count'
}, inplace=True)

# Optional: Fill any remaining NaNs with 0
forecast_df_panel['weekly_event_count'] = forecast_df_panel['weekly_event_count'].fillna(0)
forecast_df_panel['monthly_event_count'] = forecast_df_panel['monthly_event_count'].fillna(0)

In [ ]:
forecast_df_panel.info()

## Align Final Dataframe with Model Features

In [ ]:
# Set index to datetime.
forecast_df_panel = forecast_df_panel.set_index(pd.to_datetime(forecast_df_panel['date']))

required_columns = ['avg_temp', 'min_temp', 'max_temp', 'precip', 'wind_dir', 'wind_speed', 'avg_pressure']

for col in required_columns:
    if col not in forecast_df_panel.columns:
        forecast_df_panel[col] = np.nan

final_features = forecast_df_panel[best_model.feature_names_in_]
final_features.info()

In [ ]:
# Double check feature names aligned
X_test.info()

# 🔹 Monte Carlo Simulation

## Forecast Preparation

Our Gradient Boost model uses past data to understand how variables such as temperature, time of year, and location influence product demand (sold units). Then, to understand how demand might look under different future weather scenarios, we simulated possible temperature outcomes based on 30 years of historical weather data from [Meteostat](https://meteostat.net/en/) . By running our Boost model over these simulations, we are able to generate ranges of likely demand, giving us not just a single forecast — but a probability distribution with confidence intervals.

## Retrieve Climate Data

In [ ]:
# Retrieve Climate Data
temp_stats_hist = get_daily_variability()

# Check the climate data.
temp_stats_hist.info()

## Create Climate Scenarios

In [ ]:
# Run custom function to create dataframe for all temperature varibles for year 2023
future_climate_scenarios = generate_future_scenarios(temp_stats_hist, year=2023)

## Perform Monte Carlo Simulation and Forecast

In [ ]:
# Create dataframe with columns for the simulation results
predictions_2023 = run_simulation(future_climate_scenarios, forecast_df_panel, best_model, sample_uniform_monte, n_simulations=1000)

In [ ]:
predictions_2023.info()

In [ ]:
# Store 2023 predictions output as CSV
predictions_2023.to_csv('/content/drive/MyDrive/Employer Project/Datasets/Production Data/filtered_sales_temp_hols_fmatches/predictions_2023.csv', index=False)

## Visualise Model Forecast for Presentation

In this last part, we are exploring different visualisations of our model and the predictions to simplify the output for a stakeholder that we assume is not as familiar with machine learning models.

In [ ]:
predictions_2023.head()

In [ ]:
# Convert 'date' column to datetime if not already
sales_model_final['date'] = pd.to_datetime(sales_model_final['date'])

# Aggregate sold_units monthly by year and month
monthly_sales = sales_model_final.groupby(['year', 'month']).agg({
    'sold_units': 'sum'
}).reset_index()

# Create a datetime column for the first day of the month
monthly_sales['month_start'] = pd.to_datetime(monthly_sales[['year', 'month']].assign(day=1))

# Add abbreviated month name
monthly_sales['month_name'] = monthly_sales['month'].apply(lambda x: calendar.month_abbr[x])

print(monthly_sales.head())

In [ ]:
# Ensure date is datetime
predictions_2023['date'] = pd.to_datetime(predictions_2023['date'])

# Add month column
predictions_2023['month'] = predictions_2023['date'].dt.to_period('M').dt.to_timestamp()

# Aggregate total sold units per month for each scenario including 95% bounds
monthly = predictions_2023.groupby('month').agg({
    'mean': 'sum',       # Average scenario
    'low_80': 'sum',     # Pessimistic (80% lower bound)
    'up_80': 'sum',      # Optimistic (80% upper bound)
    'low_95': 'sum',     # Pessimistic (95% lower bound)
    'up_95': 'sum',      # Optimistic (95% upper bound)
}).reset_index()

print(predictions_2023.head())

In [ ]:
# Load the dataset.
predict_23 = predictions_2023.copy()

In [ ]:
# Create visual for 2023 forecast and confidence bands only
# Create 'year_month' as datetime for proper sorting
predict_23['year_month'] = pd.to_datetime(predict_23['date'].dt.to_period('M').astype(str))

# Create short month name for display (e.g., "Jan", "Feb", ...)
predict_23['month_label'] = predict_23['date'].dt.strftime('%b')

# Define aggregation functions
agg_functions = {
    'mean': 'sum',
    'low_80': 'sum',
    'up_80': 'sum',
    'low_95': 'sum',
    'up_95': 'sum',
    'forecasted_max_temp': 'mean',
    'prob_exceed_28': 'mean',
    'conditional_mean_temp': 'mean'
}

# Group by 'year_month' and aggregate
monthly_forecast = predict_23.groupby('year_month').agg(agg_functions).reset_index()

# Add month name back in for display (now matched to ordered months)
monthly_forecast['month_label'] = monthly_forecast['year_month'].dt.strftime('%b')

# Drop any rows with missing values in CI columns
monthly_forecast = monthly_forecast.dropna(subset=['low_80', 'up_80', 'low_95', 'up_95'])

# Function to format numbers in thousands with no decimals
def format_thousands(value):
    # Convert to integer after dividing by 1000 to remove decimal point
    return f'{int(value // 1000)}k' if value >= 1000 else f'{int(value)}'

def plot_summary_month(df, x_labels, title):
    fig, ax = plt.subplots(figsize=(14, 8))

    # Plot mean line
    ax.plot(x_labels, df['mean'], label='Sales Forecast')

    # Plot the fill between for confidence intervals
    ax.fill_between(x_labels, df['low_80'], df['up_80'], alpha=0.3, label='80% Confidence Interval')
    ax.fill_between(x_labels, df['low_95'], df['up_95'], alpha=0.2, label='95% Confidence Interval')

    # Add labels at the end of the confidence intervals in thousands (no decimals)
    for i, month in enumerate(x_labels):
        ax.text(i, df['up_80'][i], format_thousands(df['up_80'][i]), color='black', ha='center', va='bottom', fontsize=10)
        ax.text(i, df['up_95'][i], format_thousands(df['up_95'][i]), color='black', ha='center', va='bottom', fontsize=10)
        ax.text(i, df['mean'][i], format_thousands(df['mean'][i]), color='black', ha='center', va='bottom', fontsize=10)

    # Add titles and labels
    ax.set_title(title)
    ax.set_xlabel("Month")
    ax.set_ylabel("Units - Sales Forecast / Probability")

    # Add a custom legend for the labels (indicating units)
    ax.legend(title= "# = Thousand Units", loc='upper left')

    # Grid settings
    ax.grid(axis='x', visible=True)
    ax.grid(axis='y', visible=False)

    # Set x-axis labels and rotation
    ax.set_xticks(range(len(x_labels)))
    ax.set_xticklabels(x_labels, rotation=45, ha='right')

    # Tight layout to ensure no clipping of labels
    plt.tight_layout()
    plt.show()

plot_summary_month(monthly_forecast, monthly_forecast['month_label'].tolist(), "2023 Forecast with Confidence Bands")

In [ ]:
# Prepare historical data for 2021 and 2022
sales_model_final['date'] = pd.to_datetime(sales_model_final['date'])
historical = sales_model_final[sales_model_final['year'].isin([2021, 2022])]
historical_monthly = historical.groupby(['year', 'month']).agg({
    'sold_units': 'sum'
}).reset_index()

# Prepare predictions for 2023: mean and up_80
predictions_2023['date'] = pd.to_datetime(predictions_2023['date'])
monthly_2023 = predictions_2023.groupby(predictions_2023['date'].dt.month).agg({
    'mean': 'sum',
    'up_80': 'sum'
}).reset_index().rename(columns={'date': 'month'})

# Month abbreviations
months_abbr = [calendar.month_abbr[m] for m in range(1, 13)]

plt.figure(figsize=(14, 7))

# Plot bars for 2021 and 2022 side-by-side
bar_width = 0.35
x = np.arange(1, 13)

# Filter data for bars
data_2021 = historical_monthly[historical_monthly['year'] == 2021].sort_values('month')
data_2022 = historical_monthly[historical_monthly['year'] == 2022].sort_values('month')

# Adjusted colors
color_2021 = '#6baed6'  # Lighter blue for 2021
color_2022 = '#c6dbef'  # Even lighter for 2022

plt.bar(x - bar_width/2, data_2021['sold_units'], width=bar_width, label='2021 Actuals', color=color_2021)
plt.bar(x + bar_width/2, data_2022['sold_units'], width=bar_width, label='2022 Actuals', color=color_2022)

# Plot shaded area between mean and up_80
plt.fill_between(monthly_2023['month'], monthly_2023['mean'], monthly_2023['up_80'],
                 color='red', alpha=0.15, label='2023 Range (Avg to Upper 80%)')

# Plot mean (dashed black) and up_80 (solid red) lines with dots
plt.plot(monthly_2023['month'], monthly_2023['mean'], color='black', linestyle='--', linewidth=1.5, marker='o', label='2023 Average')
plt.plot(monthly_2023['month'], monthly_2023['up_80'], color='red', linestyle='-', linewidth=2, marker='o', label='2023 Upper 80% (recommended)')

# Add data labels with different offsets in 'k' format
mean_offset = max(monthly_2023['up_80']) * 0.01
up80_offset = max(monthly_2023['up_80']) * 0.04

for x_pos, y_val in zip(monthly_2023['month'], monthly_2023['mean']):
    plt.text(x_pos, y_val + mean_offset, f"{int(y_val/1000)}k", ha='center', va='bottom', fontsize=10)

for x_pos, y_val in zip(monthly_2023['month'], monthly_2023['up_80']):
    plt.text(x_pos, y_val + up80_offset, f"{int(y_val/1000)}k", ha='center', va='bottom', fontsize=10)

# Y-axis format: e.g., 10'000
def format_thousands(x, pos):
    return f"{int(x):,}".replace(",", "'")

plt.gca().yaxis.set_major_formatter(FuncFormatter(format_thousands))

# X-axis ticks and labels
plt.xticks(ticks=x, labels=months_abbr)
plt.ylim(0, 40000)  # <- Adjusted from 45000 to 42000
plt.xlabel('Month')
plt.ylabel('Total Units')
plt.title('Monthly Sales: 2021–2022 Actuals & 2023 Predictions')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#🔹 __Revenue Optimisation__

>
### __Data Sources and Preparation__

In [ ]:
# Locate the dataset in Google Drive.
filepath_sales_model_final = '/content/drive/MyDrive/Employer Project/Datasets/Production Data/filtered_sales_temp_hols_fmatches/sales_model_final.csv'

# Load the dataset.
sales_model_final = pd.read_csv(filepath_sales_model_final)

# Sense check the DataFrame.
sales_model_final.info()

>
### __Setting Model Parameters/Calculations__

In [ ]:
# Visualise max temp and sales to derive temperature thresholds.
plt.figure(figsize=(12, 6))

# Set bin width for temperature (2-degree bins).
bin_width = 2
bins = range(0, 44, bin_width)

# Calculate the average units sold for each temperature bin.
avg_units_per_bin = sales_model_final.groupby(pd.cut(sales_model_final['max_temp'], bins=bins))['sold_units'].mean()

# Plotthe average units sold.
plt.figure(figsize=(12, 6))
plt.bar(avg_units_per_bin.index.astype(str), avg_units_per_bin.values, color='blue', alpha=0.7, edgecolor='black')

# Add labels and title.
plt.title('Average Units Sold by Max Temperature')
plt.xlabel('Max Temperature (°C)')
plt.ylabel('Average Units Sold')

# Rotate x-axis labels for better readability.
plt.xticks(rotation=45)

# Show the plot.
plt.show()

__Observations:__ Set temperature threshold for which more units should be considered. Max temperature and previous exploration demonstrates that sales start to peak around 26-28 degrees max temp. Extreme maximum temperatures are also demonstrated as peaks around 36 to 40 (but these temperatures are not considered normal). Will set the temperature threshold to 26.

In [ ]:
# Visualise differences in temp differences from reference normals across months.
# Group the data by month and calculate the average temp_diff.
sales_model_final['temp_diff'] = (sales_model_final['max_temp'] - sales_model_final['ref_tmax'])
avg_temp_diff_per_month = sales_model_final.groupby('month')['temp_diff'].mean()

# Plotting the average temperature difference per month.
plt.figure(figsize=(14, 8))
plt.bar(avg_temp_diff_per_month.index.astype(str), avg_temp_diff_per_month.values, color='blue', alpha=0.7, edgecolor='black')

# Add labels and title.
plt.title('Average Temperature Difference by Month')
plt.xlabel('Month')
plt.ylabel('Average Temperature Difference (°C)')

# Customize x-axis to display months in order.
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.xticks(ticks=range(12), labels=months, rotation=45)

# Set y-axis to start from the minimum value in the data.
min_temp_diff = avg_temp_diff_per_month.min()
max_temp_diff = avg_temp_diff_per_month.max()
padding = 2
plt.ylim(min_temp_diff - padding, max_temp_diff + padding)

# Add a horizontal line at y=0.
plt.axhline(0, color='black', linewidth=1.2, linestyle='-')

# Show the plot
plt.show()

>
### __Requirements From the Model__

#### __Load the Predictions for 2023__

In [ ]:
# Locate the final predictions file for 2023.
filepath_2023_predictions = '/content/drive/MyDrive/Employer Project/Datasets/Production Data/filtered_sales_temp_hols_fmatches/predictions_2023.csv'

# Load the dataset.
predictions_temps_df = pd.read_csv(filepath_2023_predictions)

# Sense check the dataset.
predictions_temps_df.info()

In [ ]:
# Sense check head of DataFrame
print(predictions_temps_df[['mean', 'low_80', 'up_80', 'low_95', 'up_95', 'forecasted_max_temp', 'prob_exceed_28', 'conditional_mean_temp']].head())

#### __Load the Events Data for 2023__

In [ ]:
# Locate the final events file for 2023.
filepath_2023_events = '/content/drive/MyDrive/Employer Project/Datasets/Production Data/filtered_sales_temp_hols_fmatches/fixed_events_2023.csv'

# Load the dataset.
events_2023_df = pd.read_csv(filepath_2023_events)

# Sense check the dataset.
events_2023_df.info()

In [ ]:
# Update date columns to datetime.
events_2023_df[['start_date', 'end_date']] = events_2023_df[['start_date', 'end_date']].apply(pd.to_datetime)

# Sense-check the DataFrame.
events_2023_df

In [ ]:
# Apply a boolean is_event_day flag to the predictions DataFrame.
predictions_temps_df['is_event_day'] = predictions_temps_df['date'].apply(
    lambda x: ((events_2023_df['start_date'] <= x) & (events_2023_df['end_date'] >= x)).any()
)

# Sense-check DataFrame.
predictions_temps_df.info()

In [ ]:
# Set index to datetime.
predictions_temps_df['date'] = pd.to_datetime(predictions_temps_df['date'])
predictions_temps_df.set_index('date', inplace=True)

# Sense check DataFrame.
predictions_temps_df

>
### __Prepare Dataframe For Optimisation__

#### __Get Daily Reference Temperatures__

In [ ]:
# Retrieve reference normals.
normals_df, mean_deviation, std_deviation = get_daily_tmax_normals_mean_std(predictions_temps_df, station_id='08221', k_neighbors=5)

# Check 365 days, sense-check temperatures.
normals_df

#### __Calculate Z-Scores for Confidence Weighting__

In [ ]:
# Calculate typical deviation of historical temperatures from the reference normal
# and use z-scores to determine how anomalous a forecasted temperature is from
# the normal and assign a confidence weighting to the forecast.
optimised_forecast_df = prepare_forecast_with_z_scores(predictions_temps_df, normals_df, mean_deviation, std_deviation)

# Sense check the dataframe.
optimised_forecast_df.info()

### __Process Model Outputs__

#### __Optimise 2023 Forecast__

In [ ]:
# Optimise model outputs based on forecasted temperature, probability of heat and
# forecast confidence.
optimised_forecast_final = adjust_ratios_calc_revenue(optimised_forecast_df)

optimised_forecast_final.info()

#### __Add 2022 and 2021 Sales Figures for Comparison__

In [ ]:
# Subset original sales data for 2022 sold units.
sold_units_2022 = sales_model_final[sales_model_final['year'].isin([2022])]

# Subset 2022 data to required columns.
sold_units_2022 = sold_units_2022[['date', 'sold_units']]

# Sense-check 2022 data.
sold_units_2022.info()

In [ ]:
# Ensure 'date' is datetime
sold_units_2022['date'] = pd.to_datetime(sold_units_2022['date'])

# Aggregate to daily totals
sold_units_2022_daily = sold_units_2022.groupby('date')['sold_units'].sum().reset_index()

# Define full December date range
full_december = pd.date_range(start='2022-12-01', end='2022-12-31')

# Get existing December dates in data
existing_december_dates = sold_units_2022_daily[
    (sold_units_2022_daily['date'].dt.month == 12) &
    (sold_units_2022_daily['date'].dt.year == 2022)
]['date']

# Find missing dates
missing_dates = full_december.difference(existing_december_dates)

# Use only early December (e.g., Dec 1–15) to calculate average
early_december = sold_units_2022_daily[
    (sold_units_2022_daily['date'] >= '2022-12-01') &
    (sold_units_2022_daily['date'] <= '2022-12-15')
]
early_avg = early_december['sold_units'].mean()
adjusted_value = round(early_avg * 0.77)

# Create missing rows
missing_rows = pd.DataFrame({
    'date': missing_dates,
    'sold_units': adjusted_value
})

# Append and sort
sold_units_2022 = pd.concat([sold_units_2022_daily, missing_rows], ignore_index=True)
sold_units_2022 = sold_units_2022.sort_values('date').reset_index(drop=True)

# Preview
print(sold_units_2022.tail(10))

In [ ]:
# Define premium product parameters.
premium_ratio = 0.02
premium_price=1.89
premium_margin=0.4

# Define standard product parameters.
standard_ratio = 0.08
standard_price=1.49
standard_margin=0.2

# Calculate revenue and profit for 2022 data.
sold_units_2022['premium_revenue_2022'] = sold_units_2022['sold_units'] * premium_ratio * premium_price
sold_units_2022['standard_revenue_2022'] = sold_units_2022['sold_units'] * standard_ratio * standard_price
sold_units_2022['total_revenue_2022'] = sold_units_2022['premium_revenue_2022'] + sold_units_2022['standard_revenue_2022']
sold_units_2022['expected_profit_2022'] = (
    sold_units_2022['premium_revenue_2022'] * premium_margin +
    sold_units_2022['standard_revenue_2022'] * standard_margin
)

# Sense-check 2022 data.
sold_units_2022.info()

In [ ]:
# Subset original sales data for 2021 sold units.
sold_units_2021 = sales_model_final[sales_model_final['year'].isin([2021])]

# Subset 2021 data to required columns.
sold_units_2021 = sold_units_2021[['date', 'sold_units']]

# Calculate revenue and profit for 2021 data.
sold_units_2021['premium_revenue_2021'] = sold_units_2021['sold_units'] * premium_ratio * premium_price
sold_units_2021['standard_revenue_2021'] = sold_units_2021['sold_units'] * standard_ratio * standard_price
sold_units_2021['total_revenue_2021'] = sold_units_2021['premium_revenue_2021'] + sold_units_2021['standard_revenue_2021']
sold_units_2021['expected_profit_2021'] = (
    sold_units_2021['premium_revenue_2021'] * premium_margin +
    sold_units_2021['standard_revenue_2021'] * standard_margin
)

# Sense-check 2021 data.
sold_units_2021.info()

In [ ]:
# Prepare 2022 units for export.
# Convert 'date' to datetime.
sold_units_2022['date'] = pd.to_datetime(sold_units_2022['date'])

# Create 'month_start_date' - first day of the month for each date.
sold_units_2022['month_start_date'] = sold_units_2022['date'].values.astype('datetime64[M]')

# Aggregate by month_start_date (monthly granularity)
sold_units_2022_monthly = sold_units_2022.groupby('month_start_date').agg({
    'sold_units': 'sum',
    'premium_revenue_2022': 'sum',
    'standard_revenue_2022': 'sum',
    'total_revenue_2022': 'sum',
    'expected_profit_2022': 'sum'
}).reset_index()

# Add 'year' and 'month' columns.
sold_units_2022_monthly['year'] = sold_units_2022_monthly['month_start_date'].dt.year
sold_units_2022_monthly['month'] = sold_units_2022_monthly['month_start_date'].dt.month

In [ ]:
# View 2022 data sample.
sold_units_2022_monthly

#### __Compare 2021 and 2022 Unoptimised Sales Figures to 2023 Optimised Sales Figures__

In [ ]:
# Calculate 2023 totals.
total_sales_2023 = optimised_forecast_final['total_units'].sum()
total_revenue_2023 = optimised_forecast_final['total_revenue'].sum()
total_profit_2023 = optimised_forecast_final['expected_profit'].sum()

# Calculate 2022 totals.
total_sales_2022 = sold_units_2022['sold_units'].sum()
total_revenue_2022 = sold_units_2022['total_revenue_2022'].sum()
total_profit_2022 = sold_units_2022['expected_profit_2022'].sum()

# Calculate 2021 totals.
total_sales_2021 = sold_units_2021['sold_units'].sum()
total_revenue_2021 = sold_units_2021['total_revenue_2021'].sum()
total_profit_2021 = sold_units_2021['expected_profit_2021'].sum()

# Calculate margin of increase from 2022 to 2023.
revenue_increase_2023_2022 = total_revenue_2023 - total_revenue_2022
revenue_increase_cent_2023_2022 = (revenue_increase_2023_2022 / total_revenue_2022) * 100

profit_increase_2023_2022 = total_profit_2023 - total_profit_2022
profit_increase_cent_2023_2022 = (profit_increase_2023_2022 / total_profit_2022) * 100

# Calculate margin of increase from 2021 to 2022.
revenue_increase_2022_2021 = total_revenue_2022 - total_revenue_2021
revenue_increase_cent_2022_2021 = (revenue_increase_2022_2021 / total_revenue_2021) * 100

profit_increase_2022_2021 = total_profit_2022 - total_profit_2021
profit_increase_cent_2022_2021 = (profit_increase_2022_2021 / total_profit_2021) * 100

# View total units of 2021, 2022 & 2023
print(f'Total sold units for 2021: {total_sales_2021:.2f}')
print(f'Total sold units for 2022: {total_sales_2022:.2f}')
print(f'Total forecasted units for 2023: {total_sales_2023:.2f}')
print()

# View revenue margin increases.
print(f'Total revenue for 2021: {total_revenue_2021:.2f}')
print(f'Total revenue for 2022: {total_revenue_2022:.2f}')
print(f'Revenue increase 2022 on 2021: {revenue_increase_2022_2021:.2f}, {revenue_increase_cent_2022_2021:.2f}%\n')

print(f'Total revenue for 2022: {total_revenue_2022:.2f}')
print(f'Total revenue for 2023: {total_revenue_2023:.2f}')
print(f'Revenue increase 2023 on 2022: {revenue_increase_2023_2022:.2f}, {revenue_increase_cent_2023_2022:.2f}%\n')

# View profit margin increases.
print(f'Total profit for 2021: {total_profit_2021:.2f}')
print(f'Total profit for 2022: {total_profit_2022:.2f}')
print(f'Revenue profit 2022 on 2021: {profit_increase_2022_2021:.2f}, {profit_increase_cent_2022_2021:.2f}%\n')

print(f'Total profit for 2022: {total_profit_2022:.2f}')
print(f'Total profit for 2023: {total_profit_2023:.2f}')
print(f'Profit increase 2023 on 2022: {profit_increase_2023_2022}, {profit_increase_cent_2023_2022:.2f}%')

#### __Calculate 2023 Profit with 20% Premium Product Cap__

In [ ]:
# Run optimiser with a 20% cap for premium product.
optimised_forecast_20_cent = adjust_ratios_calc_revenue(optimised_forecast_df, max_unit2_ratio=0.20)

# Sense Check DataFrame.
optimised_forecast_20_cent.info()

In [ ]:
# Calculate 2023 totals and 20% cap for premium products.
total_sales_2023_20_cent = optimised_forecast_20_cent['total_units'].sum()
total_revenue_2023_20_cent = optimised_forecast_20_cent['total_revenue'].sum()
total_profit_2023_20_cent = optimised_forecast_20_cent['expected_profit'].sum()

# Calculate margin of increase from 2022 to 2023.
revenue_increase_2023_2022_20_cent = total_revenue_2023_20_cent - total_revenue_2022
revenue_increase_cent_2023_2022_20_cent = (revenue_increase_2023_2022_20_cent / total_revenue_2022) * 100

profit_increase_2023_2022_20_cent = total_profit_2023_20_cent - total_profit_2022
profit_increase_cent_2023_2022_20_cent = (profit_increase_2023_2022_20_cent / total_profit_2022) * 100

# View total units of 2022 & 2023
print(f'Total sold units for 2022: {total_sales_2022:.2f}')
print(f'Total forecasted units for 2023: {total_sales_2023_20_cent:.2f}')
print()

# View revenue margin increases.
print(f'Total revenue for 2022: {total_revenue_2022:.2f}')
print(f'Total revenue for 2023: {total_revenue_2023_20_cent:.2f}')
print(f'Revenue increase 2023 on 2022: {revenue_increase_2023_2022_20_cent:.2f}, {revenue_increase_cent_2023_2022_20_cent:.2f}%\n')

# View profit margin increases.
print(f'Total profit for 2022: {total_profit_2022:.2f}')
print(f'Total profit for 2023: {total_profit_2023_20_cent:.2f}')
print(f'Profit increase 2023 on 2022: {profit_increase_2023_2022_20_cent}, {profit_increase_cent_2023_2022_20_cent:.2f}%')

#### __Prepare for Dashboard Visualisation__

In [ ]:
# Sense check unique event days to crosscheck with visualisation calculations.
# Filter rows where event day is True
event_days = optimised_forecast_final[optimised_forecast_final['is_event_day']]

# Create YearMonth period.
event_days['YearMonth'] = event_days.index.to_period('M')

# Count unique event days per month.
unique_event_days_per_month = event_days.groupby('YearMonth').apply(lambda x: pd.Series(x.index.date).nunique())

# Reset index for nicer output.
unique_event_days_per_month = unique_event_days_per_month.reset_index(name='UniqueEventDays')

print(unique_event_days_per_month)

In [ ]:
# Download final DataFrames for dashboarding and sense check function outputs.
optimised_forecast_final.to_csv('/content/drive/MyDrive/Employer Project/Datasets/Production Data/filtered_sales_temp_hols_fmatches/optimised_forecast_final.csv', index=True)

sold_units_2022_monthly.to_csv('/content/drive/MyDrive/Employer Project/Datasets/Production Data/filtered_sales_temp_hols_fmatches/sold_units_2022_monthly.csv', index=False)

Revenue optimisation visualisations are located in Tableau file